# SQL Query Explanation

## Overview 

In this notebook we show the differences between the Bedrock large language models, as well as Q, when they are use to analyze the SQL queries. 


### Queries


For our test set we use the queries that are included in the Redshift Cluster review. 

### Models

In [1]:
!pwd

/Volumes/LifeSaver/LocalProjects/amazon-bedrock-samples/redshift_review_data/test


In [ ]:
queries={}

In [19]:
queries["NodeDetails"] = {}
queries["NodeDetails"]["text"] = """WITH node_slice AS
(
  SELECT node,
         COUNT(1) AS slice_count
  FROM stv_slices
  GROUP BY node
),
node_storage_utilization AS
(
  SELECT node::text AS node,
         (1.0 * used / capacity)::NUMERIC(8,4) * 100 AS storage_utilization_pct,
         1.0 * capacity/1000 AS storage_capacity_gb,
         1.0 * used/1000 AS storage_used_gb
  FROM stv_node_storage_capacity
)
SELECT CASE
         WHEN capacity = 190633 AND NOT is_nvme THEN 'dc1.large'
         WHEN capacity = 380319 THEN 'dc1.8xlarge'
         WHEN capacity = 190633 AND is_nvme THEN 'dc2.large'
         WHEN capacity = 760956 THEN 'dc2.8xlarge'
         WHEN capacity = 726296 THEN 'dc2.8xlarge'
         WHEN capacity = 952455 THEN 'ds2.xlarge'
         WHEN capacity = 945026 THEN 'ds2.8xlarge'
         WHEN capacity = 954367 AND part_count = 1 THEN 'ra3.xlplus'
         WHEN capacity = 3339176 AND part_count = 1 THEN 'ra3.4xlarge'
         WHEN capacity = 3339176 AND part_count = 4 THEN 'ra3.16xlarge'
         ELSE 'unknown'
       END AS node_type,
       s.node,
       slice_count,
       storage_utilization_pct,
       storage_capacity_gb,
       storage_used_gb
FROM (SELECT p.host AS node,
             p.capacity,
             p.mount LIKE '/dev/nvme%' AS is_nvme,
             COUNT(1) AS part_count
      FROM stv_partitions p
      WHERE p.host = p.owner
      GROUP BY 1,
               2,
               3) AS s
  INNER JOIN node_slice n ON (s.node = n.node)
  INNER JOIN node_storage_utilization ns ON (s.node = ns.node)
ORDER by 2;"""

In [20]:
queries["WLMConfig"] = {}
queries["WLMConfig"]["text"] = """SELECT c.wlm_mode,
       scc.service_class::text AS service_class_id,
       CASE
         WHEN scc.service_class BETWEEN 1 AND 4 THEN 'System'
         WHEN scc.service_class = 5 THEN 'Superuser'
         WHEN scc.service_class BETWEEN 6 AND 13 THEN 'Manual WLM'
         WHEN scc.service_class = 14 THEN 'SQA'
         WHEN scc.service_class = 15 THEN 'Redshift Maintenance'
         WHEN scc.service_class BETWEEN 100 AND 107 THEN 'Auto WLM'
       END AS service_class_category,
       trim(scc.name) AS queue_name,
       CASE
         WHEN scc.num_query_tasks = -1 THEN 'auto'
         ELSE scc.num_query_tasks::text
       END AS slots,
       CASE
         WHEN scc.query_working_mem = -1 THEN 'auto'
         ELSE scc.query_working_mem::text
       END AS query_working_memory_mb_per_slot,
       nvl(cast(ROUND(((scc.num_query_tasks*scc.query_working_mem)::NUMERIC/ mem.total_memory_mb::NUMERIC)*100,0)::NUMERIC(38,4) as varchar(12)),'auto') cluster_memory_pct,
       scc.max_execution_time AS query_timeout,
       trim(scc.concurrency_scaling) AS concurrency_scaling,
       trim(scc.query_priority) AS queue_priority,
       nvl(qc.qmr_rule_count,0) AS qmr_rule_count,
       CASE
         WHEN qmr.qmr_rule IS NOT NULL THEN 'Y'
         ELSE 'N'
       END AS is_queue_evictable,
       LISTAGG(DISTINCT TRIM(qmr.qmr_rule),',') within group(ORDER BY rule_name) qmr_rule,
       LISTAGG(TRIM(cnd.condition),', ') condition
FROM stv_wlm_service_class_config scc
  INNER JOIN stv_wlm_classification_config cnd ON scc.service_class = cnd.action_service_class
  CROSS JOIN (SELECT CASE
                       WHEN COUNT(1) > 0 THEN 'auto'
                       ELSE 'manual'
                     END AS wlm_mode
              FROM stv_wlm_service_class_config
              WHERE service_class >= 100) c
  CROSS JOIN (SELECT SUM(num_query_tasks*query_working_mem) AS total_memory_mb
              FROM stv_wlm_service_class_config
              WHERE service_class BETWEEN 6 AND 13) mem
  LEFT OUTER JOIN (SELECT service_class,
                          COUNT(DISTINCT rule_name) AS qmr_rule_count
                   FROM stv_wlm_qmr_config
                   GROUP BY service_class) qc ON (scc.service_class = qc.service_class)
  LEFT OUTER JOIN (SELECT service_class,
                          rule_name,
                          rule_name || ':' || '[' || action || '] ' || metric_name || metric_operator || CAST(metric_value AS VARCHAR(256)) qmr_rule
                   FROM stv_wlm_qmr_config) qmr ON scc.service_class = qmr.service_class
WHERE scc.service_class > 4
GROUP BY 1,
         2,
         3,
         4,
         5,
         6,
         7,
         8,
         9,
         10,
         11,
         12
ORDER BY 2 ASC;"""

In [21]:
queries["WLMandCommit"] = {}
queries["WLMandCommit"]["text"] = """SELECT IQ.*,
       (IQ.wlm_queue_time_ms/ IQ.wlm_start_commit_time_ms)*100.0::numeric(6,2) AS pct_wlm_queue_time,
       (IQ.exec_only_time_ms/ IQ.wlm_start_commit_time_ms)*100.0::numeric(6,2) AS pct_exec_only_time,
       (IQ.commit_queue_time_ms/ IQ.wlm_start_commit_time_ms)*100.0::numeric(6,2) pct_commit_queue_time,
       (IQ.commit_time_ms/ IQ.wlm_start_commit_time_ms)*100.0::numeric(6,2) pct_commit_time
FROM (SELECT TRUNC(b.starttime) AS DAY,
             d.service_class,
             rtrim(s.name) as queue_name,
             c.node,
             COUNT(DISTINCT c.xid) AS count_commit_xid,
             SUM(datediff ('microsec',d.service_class_start_time,c.endtime)*0.001)::numeric(38,4) AS wlm_start_commit_time_ms,
             SUM(datediff ('microsec',d.queue_start_time,d.queue_end_time)*0.001)::numeric(38,4) AS wlm_queue_time_ms,
             SUM(datediff ('microsec',b.starttime,b.endtime)*0.001)::numeric(38,4) AS exec_only_time_ms,
             SUM(datediff ('microsec',c.startwork,c.endtime)*0.001)::numeric(38,4) commit_time_ms,
             SUM(datediff ('microsec',DECODE(c.startqueue,'2000-01-01 00:00:00',c.startwork,c.startqueue),c.startwork)*0.001)::numeric(38,4) commit_queue_time_ms
      FROM stl_query b,
           stl_commit_stats c,
           stl_wlm_query d,
           stv_wlm_service_class_config s
      WHERE b.xid = c.xid
      AND   b.query = d.query
      AND   c.xid > 0
      AND d.service_class = s.service_class
      GROUP BY 1,2,3,4
      ORDER BY 1,2,3,4) IQ;"""

In [22]:
queries["QueueHealth"] = {}
queries["QueueHealth"]["text"] = """with workload as
(
select trim(sq."database") as dbname
      ,case 
	     when sq.concurrency_scaling_status = 1 then 'burst'	        
			 else 'main' end as concurrency_scaling_status 
      ,case 
       when sl.source_query is not null then 'result_cache'       
			 else rtrim(swsc.name) end as queue_name 
			,swq.service_class 
      ,case
       when swq.service_class between 1 and 4 then 'System'
       when swq.service_class = 5 then 'Superuser'
       when swq.service_class between 6 and 13 then'Manual WLM queues'
       when swq.service_class = 14 then 'SQA'
       when swq.service_class = 15 then 'Redshift Maintenance'
       when swq.service_class between 100 and 107 then 'Auto WLM'
       end as service_class_category 	  
      ,sq.query as query_id
      ,case 
         when regexp_instr(sq.querytxt, '(padb_|pg_internal)'             ) then 'OTHER'
         when regexp_instr(sq.querytxt, '([uU][nN][dD][oO][iI][nN][gG]) ' ) then 'SYSTEM'
         when regexp_instr (sq.querytxt,'([aA][uU][tT][oO][mM][vV])'      ) then 'AUTOMV'
         when regexp_instr(sq.querytxt, '[uU][nN][lL][oO][aA][dD]'        ) then 'UNLOAD'
         when regexp_instr(sq.querytxt, '[cC][uU][rR][sS][oO][rR] '       ) then 'CURSOR'
         when regexp_instr(sq.querytxt, '[fF][eE][tT][cC][hH] '           ) then 'CURSOR'
         WHEN regexp_instr (sq.querytxt,'[cC][rR][eE][aA][tT][eE] '       ) then 'CTAS'
         when regexp_instr(sq.querytxt, '[dD][eE][lL][eE][tT][eE] '       ) then 'DELETE'
         when regexp_instr(sq.querytxt, '[uU][pP][dD][aA][tT][eE] '       ) then 'UPDATE'
         when regexp_instr(sq.querytxt, '[iI][nN][sS][eE][rR][tT] '       ) then 'INSERT'
         when regexp_instr(sq.querytxt, '[vV][aA][cC][uU][uU][mM][ :]'    ) then 'VACUUM'
         when regexp_instr(sq.querytxt, '[aA][nN][aA][lL][yY][zZ][eE] '   ) then 'ANALYZE'		 
         when regexp_instr(sq.querytxt, '[sS][eE][lL][eE][cC][tT] '       ) then 'SELECT'
         when regexp_instr(sq.querytxt, '[cC][oO][pP][yY] '               ) then 'COPY'
         else 'OTHER' 
       end as query_type 
      ,date_trunc('hour',sq.starttime) as workload_exec_hour
      ,nvl(swq.est_peak_mem/1024.0/1024.0/1024.0,0.0) as est_peak_mem_gb
      ,decode(swq.final_state, 'Completed',decode(swr.action, 'abort',0,decode(sq.aborted,0,1,0)),'Evicted',0,null,decode(sq.aborted,0,1,0)::int) as is_completed
      ,decode(swq.final_state, 'Completed',decode(swr.action, 'abort',1,0),'Evicted',1,null,0) as is_evicted_aborted
      ,decode(swq.final_state, 'Completed',decode(swr.action, 'abort',0,decode(sq.aborted,1,1,0)),'Evicted',0,null,decode(sq.aborted,1,1,0)::int) as is_user_aborted
	    ,case when sl.from_sp_call is not null then 1 else 0 end as from_sp_call
	    ,case when alrt.num_events is null then 0 else alrt.num_events end as alerts
	    ,case when dsk.num_diskbased > 0 then 1 else 0 end as is_query_diskbased
	    ,nvl(c.num_compile_segments,0) as num_compile_segments
      ,cast(case when sqms.query_queue_time is null then 0 else sqms.query_queue_time end as decimal(26,6)) as query_queue_time_secs
	    ,nvl(c.max_compile_time_secs,0) as max_compile_time_secs
	    ,sl.starttime
	    ,sl.endtime
	    ,sl.elapsed
      ,cast(sl.elapsed * 0.000001 as decimal(26,6)) as query_execution_time_secs	
      ,sl.elapsed * 0.000001 - nvl(c.max_compile_time_secs,0)  - nvl(sqms.query_queue_time,0) as actual_execution_time_secs	  
      ,case when sqms.query_temp_blocks_to_disk is null then 0 else sqms.query_temp_blocks_to_disk end as query_temp_blocks_to_disk_mb
      ,cast(case when sqms.query_cpu_time is null then 0 else sqms.query_cpu_time end as decimal(26,6)) as query_cpu_time_secs 
	    ,nvl(sqms.scan_row_count,0) as scan_row_count
      ,nvl(sqms.return_row_count,0) as return_row_count
      ,nvl(sqms.nested_loop_join_row_count,0) as nested_loop_join_row_count
      ,nvl(uc.usage_limit_count,0) as cs_usage_limit_count
  from stl_query sq
  inner join svl_qlog sl on (sl.userid = sq.userid and sl.query = sq.query)
  left outer join svl_query_metrics_summary sqms on (sqms.userid = sq.userid and sqms.query = sq.query)					
  left outer join stl_wlm_query swq on (sq.userid = swq.userid and sq.query = swq.query)
  left outer join stl_wlm_rule_action swr on (sq.userid = swr.userid and sq.query = swr.query and swq.service_class = swr.service_class)
  left outer join stv_wlm_service_class_config swsc on (swsc.service_class = swq.service_class)
  left outer join (select sae.query
                         ,cast(1 as integer) as num_events
                     from svcs_alert_event_log sae
                   group by sae.query) as alrt on (alrt.query = sq.query)  
  left outer join (select sqs.userid
                         ,sqs.query
                         ,1 as num_diskbased
                     from svcs_query_summary sqs    
                    where sqs.is_diskbased = 't'
                   group by sqs.userid, sqs.query
                   ) as dsk on (dsk.userid = sq.userid and dsk.query = sq.query)  
  left outer join (select userid, xid,  pid, query
                         ,max(datediff(ms, starttime, endtime)*1.0/1000) as max_compile_time_secs
	                     ,sum(compile) as num_compile_segments
                     from svcs_compile
                   group by userid, xid,  pid, query
                  ) c on (c.userid = sq.userid and c.xid = sq.xid and c.pid = sq.pid and c.query = sq.query)                 
  left outer join (select query,xid,pid
                          ,count(1) as usage_limit_count
                      from stl_usage_control 
                     where feature_type = 'CONCURRENCY_SCALING'
                   group by query, xid, pid) uc on (uc.xid = sq.xid and uc.pid = sq.pid and uc.query = sq.query)                  	   
  where sq.userid <> 1 
    and sq.querytxt not like 'padb_fetch_sample%'
    and sq.starttime >= dateadd(day,-7,current_date)
)
select workload_exec_hour
      ,service_class_category
      ,service_class
      ,queue_name
      ,concurrency_scaling_status
      ,dbname
      ,query_type
      ,sum(is_completed) + sum(is_user_aborted) + sum(is_evicted_aborted) as total_query_count
      ,sum(is_completed) as completed_query_count
      ,sum(is_user_aborted) as user_aborted_count
      ,sum(is_evicted_aborted) as wlm_evicted_count
      ,round(sum(est_peak_mem_gb),4) as total_est_peak_mem_gb
      ,sum(is_query_diskbased) as total_disk_spill_count
      ,sum(num_compile_segments) as total_compile_count
      ,round(sum(query_temp_blocks_to_disk_mb/1024.0),4) as total_disk_spill_gb
      ,sum(alerts) as total_query_alert_count
      ,sum(from_sp_call) as total_called_proc_count
	  ,avg(query_execution_time_secs) as avg_query_execution_time_secs
	  ,max(query_execution_time_secs) as max_query_execution_time_secs
      ,sum(query_execution_time_secs) as total_query_execution_time_secs
	  ,avg(max_compile_time_secs) as avg_compile_time_secs
	  ,max(max_compile_time_secs) as max_compile_time_secs
      ,sum(max_compile_time_secs) as total_compile_time_secs
	  ,avg(query_queue_time_secs) as avg_query_queue_time_secs
	  ,max(query_queue_time_secs) as max_query_queue_time_secs
      ,sum(query_queue_time_secs) as total_query_queue_time_secs
	  ,avg(actual_execution_time_secs) as avg_actual_execution_time_secs
      ,sum(actual_execution_time_secs) as total_actual_execution_time_secs
      ,sum(query_cpu_time_secs) as total_query_cpu_time_secs
      ,sum(cs_usage_limit_count) as total_cs_usage_limit_count
      ,sum(scan_row_count) as total_scan_row_count
      ,sum(return_row_count) as total_return_row_count
      ,sum(nested_loop_join_row_count) as total_nl_join_row_count
  from workload
  group by 1,2,3,4,5,6,7
  order by 1,2,3,4,5,6,7;"""

In [23]:
queries["UsagePattern"] = {}
queries["UsagePattern"]["text"] = """WITH workload_profile AS
(
  SELECT trim(t."database") AS dbname,
         s.statement_type,
         CAST(CASE 
              WHEN regexp_instr (s.statement_text,'(padb_|pg_internal)'              ) THEN 'SYSTEM' 
              WHEN regexp_instr (s.statement_text,'([uU][nN][dD][oO][iI][nN][gG]) '  ) THEN 'SYSTEM' 
              WHEN regexp_instr (s.statement_text,'([aA][uU][tT][oO][mM][vV])'       ) THEN 'AUTOMV' 
              WHEN regexp_instr (s.statement_text,'[uU][nN][lL][oO][aA][dD]'         ) THEN 'UNLOAD' 
              WHEN regexp_instr (s.statement_text,'[cC][uU][rR][sS][oO][rR] '        ) THEN 'CURSOR' 
              WHEN regexp_instr (s.statement_text,'[cC][rR][eE][aA][tT][eE] '        ) THEN 'DDL' 
              WHEN regexp_instr (s.statement_text,'[aA][lL][tT][eE][rR] '            ) THEN 'DDL' 
              WHEN regexp_instr (s.statement_text,'[dD][rR][oO][pP] '                ) THEN 'DDL' 
              WHEN regexp_instr (s.statement_text,'[tT][rR][uU][nN][cC][aA][tT][eE] ') THEN 'DDL' 
              WHEN regexp_instr (s.statement_text,'[cC][aA][lL][lL] '                ) THEN 'CALL_PROCEDURE' 
              WHEN regexp_instr (s.statement_text,'[gG][rR][aA][nN][tT] '            ) THEN 'DCL' 
              WHEN regexp_instr (s.statement_text,'[rR][eE][vV][oO][kK][eE] '        ) THEN 'DCL' 
              WHEN regexp_instr (s.statement_text,'[fF][eE][tT][cC][hH] '            ) THEN 'CURSOR' 
              WHEN regexp_instr (s.statement_text,'[dD][eE][lL][eE][tT][eE] '        ) THEN 'DELETE'
              WHEN regexp_instr (s.statement_text,'[uU][pP][dD][aA][tT][eE] '        ) THEN 'UPDATE' 
              WHEN regexp_instr (s.statement_text,'[iI][nN][sS][eE][rR][tT] '        ) THEN 'INSERT' 
              WHEN regexp_instr (s.statement_text,'[vV][aA][cC][uU][uU][mM][ :]'     ) THEN 'VACUUM' 
              WHEN regexp_instr (s.statement_text,'[aA][nN][aA][lL][yY][zZ][eE] '    ) THEN 'ANALYZE' 
              WHEN regexp_instr (s.statement_text,'[sS][eE][lL][eE][cC][tT] '        ) THEN 'SELECT' 
              WHEN regexp_instr (s.statement_text,'[cC][oO][pP][yY] '                ) THEN 'COPY' 
              ELSE 'OTHER' 
              END AS VARCHAR(32)) AS query_type,
         DATEPART(HOUR,t.starttime) query_hour,
		 t.starttime::DATE as query_date,
         ROUND(SUM(duration_secs),1) query_duration_secs,
         COUNT(*) query_count
  FROM (SELECT s.userid,
               s.type AS statement_type,
               s.xid,
               s.pid,
               DATEDIFF(milliseconds,starttime,endtime)::NUMERIC(38,4) / 1000 AS duration_secs,
               CAST(REPLACE(rtrim(s.text),'\n','') AS VARCHAR(800)) AS statement_text
        FROM svl_statementtext s
        WHERE s.sequence = 0
        AND   s.starttime >= DATEADD(DAY,-7,CURRENT_DATE)
       ) s
    LEFT OUTER JOIN stl_query t
                 ON (s.xid = t.xid
                AND s.pid = t.pid
                AND s.userid = t.userid)
  WHERE s.statement_text NOT LIKE '%volt_tt_%'
  GROUP BY 1,2,3,4,5
)
SELECT wp.dbname,
       wp.query_date,
       wp.query_hour,
       MAX(NVL(CASE WHEN wp.query_type = 'SELECT' THEN query_count ELSE 0 END,0))::bigint AS select_count,
       MAX(NVL(CASE WHEN wp.query_type = 'SELECT' THEN query_duration_secs ELSE 0 END,0)) AS select_duration_secs,
       MAX(NVL(CASE WHEN wp.query_type = 'CURSOR' THEN query_count ELSE 0 END,0))::bigint AS cursor_count,
       MAX(NVL(CASE WHEN wp.query_type = 'CURSOR' THEN query_duration_secs ELSE 0 END,0)) AS cursor_duration_secs,
       MAX(NVL(CASE WHEN wp.query_type = 'UNLOAD' THEN query_count ELSE 0 END,0))::bigint AS unload_count,
       MAX(NVL(CASE WHEN wp.query_type = 'UNLOAD' THEN query_duration_secs ELSE 0 END,0)) AS unload_duration_secs,
       MAX(NVL(CASE WHEN wp.query_type = 'COPY' THEN query_count ELSE 0 END,0))::bigint AS copy_count,
       MAX(NVL(CASE WHEN wp.query_type = 'COPY' THEN query_duration_secs ELSE 0 END,0)) AS copy_duration_secs,
       MAX(NVL(CASE WHEN wp.query_type = 'INSERT' THEN query_count ELSE 0 END,0))::bigint AS insert_count,
       MAX(NVL(CASE WHEN wp.query_type = 'INSERT' THEN query_duration_secs ELSE 0 END,0)) AS insert_duration_secs,
       MAX(NVL(CASE WHEN wp.query_type = 'UPDATE' THEN query_count ELSE 0 END,0))::bigint AS update_count,
       MAX(NVL(CASE WHEN wp.query_type = 'UPDATE' THEN query_duration_secs ELSE 0 END,0)) AS update_duration_secs,
       MAX(NVL(CASE WHEN wp.query_type = 'DELETE' THEN query_count ELSE 0 END,0))::bigint AS delete_count,
       MAX(NVL(CASE WHEN wp.query_type = 'DELETE' THEN query_duration_secs ELSE 0 END,0)) AS delete_duration_secs,
       MAX(NVL(CASE WHEN wp.query_type = 'VACUUM' THEN query_count ELSE 0 END,0))::bigint AS vacuum_count,
       MAX(NVL(CASE WHEN wp.query_type = 'VACUUM' THEN query_duration_secs ELSE 0 END,0)) AS vacuum_duration_secs,
       MAX(NVL(CASE WHEN wp.query_type = 'ANALYZE' THEN query_count ELSE 0 END,0))::bigint AS analyze_count,
       MAX(NVL(CASE WHEN wp.query_type = 'ANALYZE' THEN query_duration_secs ELSE 0 END,0)) AS analyze_duration_secs,
       MAX(NVL(CASE WHEN wp.query_type = 'CALL_PROCEDURE' THEN query_count ELSE 0 END,0))::bigint AS call_proc_count,
       MAX(NVL(CASE WHEN wp.query_type = 'CALL_PROCEDURE' THEN query_duration_secs ELSE 0 END,0)) AS call_proc_duration_secs,
       MAX(NVL(CASE WHEN wp.query_type = 'DDL' THEN query_count ELSE 0 END,0))::bigint AS ddl_count,
       MAX(NVL(CASE WHEN wp.query_type = 'DDL' THEN query_duration_secs ELSE 0 END,0)) AS ddl_duration_secs,
       MAX(NVL(CASE WHEN wp.query_type = 'DCL' THEN query_count ELSE 0 END,0))::bigint AS dcl_count,
       MAX(NVL(CASE WHEN wp.query_type = 'DCL' THEN query_duration_secs ELSE 0 END,0)) AS dcl_duration_secs,
       MAX(NVL(CASE WHEN wp.query_type = 'AUTOMV' THEN query_count ELSE 0 END,0))::bigint AS automv_count,
       MAX(NVL(CASE WHEN wp.query_type = 'AUTOMV' THEN query_duration_secs ELSE 0 END,0)) AS automv_duration_secs,
       MAX(NVL(CASE WHEN wp.query_type = 'OTHER' THEN query_count ELSE 0 END,0))::bigint AS other_count,
       MAX(NVL(CASE WHEN wp.query_type = 'OTHER' THEN query_duration_secs ELSE 0 END,0)) AS other_duration_secs
FROM workload_profile wp
WHERE wp.dbname <> 'padb_harvest'
GROUP BY 1,2,3
ORDER BY 1,2,3;"""

In [24]:
queries["ConcurrencyScalingUsage"] = {}
queries["ConcurrencyScalingUsage"]["text"] = """select date_trunc('hour',end_time) as burst_hour
      ,sum(queries) as query_count
      ,sum(usage_in_seconds) as burst_usage_in_seconds  
from svcs_concurrency_scaling_usage
group by burst_hour;"""

In [25]:
queries["TableAlerts"] = {}
queries["TableAlerts"]["text"] = """SELECT trim(q."database") as dbname,
       trim(s.perm_table_name) AS table_name,
       COALESCE(SUM(ABS(datediff (microsecond,COALESCE(b.starttime,d.starttime,s.starttime),CASE WHEN COALESCE(b.endtime,d.endtime,s.endtime) >COALESCE(b.starttime,d.starttime,s.starttime) THEN COALESCE(b.endtime,d.endtime,s.endtime) ELSE COALESCE(b.starttime,d.starttime,s.starttime) END)))/ 1000000::NUMERIC(38,4),0) AS alert_seconds,
       COALESCE(SUM(COALESCE(b.rows,d.rows,s.rows)),0) AS alert_rowcount,
       trim(split_part (l.event,':',1)) AS alert_event,
       substring(trim(l.solution),1,200) AS alert_solution,
       MAX(l.query) AS alert_sample_query,
       COUNT(DISTINCT l.query) alert_querycount
FROM stl_alert_event_log AS l
  LEFT JOIN stl_scan AS s
         ON s.query = l.query
        AND s.slice = l.slice
        AND s.segment = l.segment
        AND s.userid > 1
        AND s.perm_table_name NOT IN ('Internal Worktable','S3')
		AND s.perm_table_name NOT LIKE ('volt_tt%')
		AND s.perm_table_name NOT LIKE ('mv_tbl__auto_mv%')			
  LEFT JOIN stl_dist AS d
         ON d.query = l.query
        AND d.slice = l.slice
        AND d.segment = l.segment
        AND d.userid > 1
  LEFT JOIN stl_bcast AS b
         ON b.query = l.query
        AND b.slice = l.slice
        AND b.segment = l.segment
        AND b.userid > 1    
  LEFT JOIN stl_query AS q   
         ON q.query = l.query
        AND q.xid = l.xid
        AND q.userid > 1           
WHERE l.userid > 1
  AND trim(s.perm_table_name) IS NOT NULL
  AND l.event_time >= dateadd(day,- 7,CURRENT_DATE)
GROUP BY 1,2,5,6
ORDER BY alert_seconds DESC;"""

In [26]:
queries["TableInfo"] = {}
queries["TableInfo"]["text"] ="""SELECT t."database" AS dbname,
       t."schema" AS namespace,
       t."table" table_name,
       t.encoded,
       t.diststyle,
       t.sortkey1,
       t.max_varchar,
       trim(t.sortkey1_enc) AS sortkey1_enc,
       t.sortkey_num,
       t.unsorted,
       t.stats_off,
       t.tbl_rows,
       t.skew_rows,
       t.estimated_visible_rows,
       CASE
         WHEN t.tbl_rows - t.estimated_visible_rows < 0 THEN 0
         ELSE (t.tbl_rows - t.estimated_visible_rows)
       END AS num_rows_marked_for_deletion,
       CASE
         WHEN t.tbl_rows - t.estimated_visible_rows < 0 THEN 0
         ELSE (t.tbl_rows - t.estimated_visible_rows) /
           CASE
             WHEN nvl (t.tbl_rows,0) = 0 THEN 1
             ELSE t.tbl_rows
           END ::NUMERIC(38,4)
       END AS pct_rows_marked_for_deletion,
       t.vacuum_sort_benefit,
       v.vacuum_run_type,
       v.is_last_vacuum_recluster,
       v.last_vacuumed_date,
       v.days_since_last_vacuumed,
       NVL(s.num_qs,0) query_count,
       nvl(sat.table_recommendation_count,0) AS table_recommendation_count,
       c.encoded_column_count,
       c.column_count,
       c.encoded_column_pct::NUMERIC(38,4) AS encoded_column_pct,
       c.encoded_sortkey_count,
       c.distkey_column_count,
       nvl(tc.large_column_size_count,0) AS large_column_size_count,
       tak.alert_sample_query AS sort_key_alert_sample_query,
       nvl(tak.alert_query_count,0) AS sort_key_alert_query_count,
       tas.alert_sample_query AS stats_alert_sample_query,
       nvl(tas.alert_query_count,0) AS stats_alert_query_count,
       tanl.alert_sample_query AS nl_alert_sample_query,
       nvl(tanl.alert_query_count,0) AS nl_alert_query_count,
       tad.alert_sample_query AS distributed_alert_sample_query,
       nvl(tad.alert_query_count,0) AS distributed_alert_query_count,
       tab.alert_sample_query AS distributed_alert_sample_query,
       nvl(tab.alert_query_count,0) AS broadcasted_alert_query_count,
       tax.alert_sample_query AS deleted_alert_sample_query,
       nvl(tax.alert_query_count,0) AS deleted_alert_query_count
FROM SVV_TABLE_INFO t
  INNER JOIN (SELECT attrelid,
                     COUNT(1) column_count,
                     SUM(CASE WHEN attisdistkey = FALSE THEN 0 ELSE 1 END) AS distkey_column_count,
                     SUM(CASE WHEN attencodingtype IN (0,128) THEN 0 ELSE 1 END) AS encoded_column_count,
                     1.0 *SUM(CASE WHEN attencodingtype IN (0,128) THEN 0 ELSE 1 END) / COUNT(1)*100 encoded_column_pct,
                     SUM(CASE WHEN attencodingtype NOT IN (0,128) AND attsortkeyord > 0 THEN 1 ELSE 0 END) AS encoded_sortkey_count
              FROM pg_attribute
              WHERE attnum > 0
              GROUP BY attrelid) c ON (c.attrelid = t.table_id)
  LEFT OUTER JOIN (SELECT tbl,
                          perm_table_name,
                          COUNT(DISTINCT query) num_qs
                   FROM stl_scan s
                   WHERE s.userid > 1
                   AND   s.perm_table_name NOT IN ('Internal Worktable','S3')
                   GROUP BY 1,
                            2) s ON (s.tbl = t.table_id)
  LEFT OUTER JOIN (SELECT DATABASE,
                          table_id,
                          COUNT(1) AS table_recommendation_count
                   FROM svv_alter_table_recommendations
                   GROUP BY DATABASE,
                            table_id) sat
               ON (sat.table_id = t.table_id
              AND sat.database = t.database)
  LEFT OUTER JOIN (SELECT sc.table_catalog AS database_name,
                          sc.table_schema,
                          sc.table_name,
                          SUM(CASE WHEN sc.character_maximum_length > 1000 THEN 1 ELSE 0 END) AS large_column_size_count
                   FROM svv_columns sc
                     INNER JOIN svv_table_info st
                             ON (sc.table_catalog = st.database
                            AND sc.table_schema = st.schema
                            AND sc.table_name = st.table)
                   WHERE sc.data_type IN ('character varying','character')
                   AND   sc.character_maximum_length > 1000
                   AND   sc.table_schema NOT IN ('pg_internal','pg_catalog','pg_automv')
                   GROUP BY sc.table_catalog,
                            sc.table_schema,
                            sc.table_name) tc
               ON (t.database = tc.database_name
              AND tc.table_name = t.table
              AND tc.table_schema = t.schema)
  LEFT OUTER JOIN (SELECT t.database AS database_name,
                          t.schema AS schema_name,
                          t.table AS table_name,
                          v.table_id,
                          CASE
                            WHEN v.status LIKE '%VacuumBG%' THEN 'Automatic'
                            ELSE 'Manual'
                          END AS vacuum_run_type,
                          CASE
                            WHEN v.is_recluster = 0 THEN 'N'
                            WHEN v.is_recluster = 1 THEN 'Y'
                            ELSE NULL
                          END AS is_last_vacuum_recluster,
                          CAST(MAX(v.eventtime) AS DATE) AS last_vacuumed_date,
                          datediff(d,CAST(MAX(v.eventtime) AS DATE),CAST(CURRENT_TIMESTAMP AT TIME ZONE 'UTC' AS DATE)) AS days_since_last_vacuumed
                   FROM stl_vacuum v
                     INNER JOIN svv_table_info t ON (t.table_id = v.table_id)
                   WHERE v.status NOT LIKE '%Started%'
                   GROUP BY 1,
                            2,
                            3,
                            4,
                            5,
                            6) v
               ON (v.database_name = t.database
              AND v.table_name = t.table
              AND v.schema_name = t.schema)
  LEFT OUTER JOIN (SELECT q.database AS database_name,
                          TRIM(s.perm_table_name) AS table_name,
                          MAX(l.query) AS alert_sample_query,
                          COUNT(DISTINCT l.query) AS alert_query_count
                   FROM stl_alert_event_log AS l
                     LEFT JOIN stl_scan AS s
                            ON s.query = l.query
                           AND s.slice = l.slice
                           AND s.segment = l.segment
                           AND s.userid > 1
                           AND s.perm_table_name NOT IN ('Internal Worktable', 'S3') 
                     LEFT JOIN stl_query AS q
                            ON q.query = l.query
                           AND q.xid = l.xid
                           AND q.userid > 1
                   WHERE l.userid > 1
                   AND   TRIM(s.perm_table_name) IS NOT NULL
                   AND   l.event_time >= dateadd(DAY,- 7,CURRENT_DATE)
                   AND   TRIM(split_part(l.event,':',1)) = 'Very selective query filter'
                   GROUP BY 1,
                            2) tak
               ON (tak.database_name = t.database
              AND tak.table_name = t.table)
  LEFT OUTER JOIN (SELECT q.database AS database_name,
                          TRIM(s.perm_table_name) AS table_name,
                          MAX(l.query) AS alert_sample_query,
                          COUNT(DISTINCT l.query) AS alert_query_count
                   FROM stl_alert_event_log AS l
                     LEFT JOIN stl_scan AS s
                            ON s.query = l.query
                           AND s.slice = l.slice
                           AND s.segment = l.segment
                           AND s.userid > 1
                           AND s.perm_table_name NOT IN ('Internal Worktable', 'S3') 
                     LEFT JOIN stl_query AS q
                            ON q.query = l.query
                           AND q.xid = l.xid
                           AND q.userid > 1
                   WHERE l.userid > 1
                   AND   TRIM(s.perm_table_name) IS NOT NULL
                   AND   l.event_time >= dateadd(DAY,- 7,CURRENT_DATE)
                   AND   TRIM(split_part(l.event,':',1)) = 'Missing query planner statistics'
                   GROUP BY 1,
                            2) tas
               ON (tas.database_name = t.database
              AND tas.table_name = t.table)
  LEFT OUTER JOIN (SELECT q.database AS database_name,
                          TRIM(s.perm_table_name) AS table_name,
                          MAX(l.query) AS alert_sample_query,
                          COUNT(DISTINCT l.query) AS alert_query_count
                   FROM stl_alert_event_log AS l
                     LEFT JOIN stl_scan AS s
                            ON s.query = l.query
                           AND s.slice = l.slice
                           AND s.segment = l.segment
                           AND s.userid > 1
                           AND s.perm_table_name NOT IN ('Internal Worktable', 'S3') 
                     LEFT JOIN stl_query AS q
                            ON q.query = l.query
                           AND q.xid = l.xid
                           AND q.userid > 1
                   WHERE l.userid > 1
                   AND   TRIM(s.perm_table_name) IS NOT NULL
                   AND   l.event_time >= dateadd(DAY,- 7,CURRENT_DATE)
                   AND   TRIM(split_part(l.event,':',1)) = 'Nested Loop Join in the query plan'
                   GROUP BY 1,
                            2) tanl
               ON (tanl.database_name = t.database
              AND tanl.table_name = t.table)
  LEFT OUTER JOIN (SELECT q.database AS database_name,
                          TRIM(s.perm_table_name) AS table_name,
                          MAX(l.query) AS alert_sample_query,
                          COUNT(DISTINCT l.query) AS alert_query_count
                   FROM stl_alert_event_log AS l
                     LEFT JOIN stl_scan AS s
                            ON s.query = l.query
                           AND s.slice = l.slice
                           AND s.segment = l.segment
                           AND s.userid > 1
                           AND s.perm_table_name NOT IN ('Internal Worktable', 'S3') 
                     LEFT JOIN stl_query AS q
                            ON q.query = l.query
                           AND q.xid = l.xid
                           AND q.userid > 1
                   WHERE l.userid > 1
                   AND   TRIM(s.perm_table_name) IS NOT NULL
                   AND   l.event_time >= dateadd(DAY,- 7,CURRENT_DATE)
                   AND   TRIM(split_part(l.event,':',1)) = 'Distributed a large number of rows across the network'
                   GROUP BY 1,
                            2) tad
               ON (tad.database_name = t.database
              AND tad.table_name = t.table)
  LEFT OUTER JOIN (SELECT q.database AS database_name,
                          TRIM(s.perm_table_name) AS table_name,
                          MAX(l.query) AS alert_sample_query,
                          COUNT(DISTINCT l.query) AS alert_query_count
                   FROM stl_alert_event_log AS l
                     LEFT JOIN stl_scan AS s
                            ON s.query = l.query
                           AND s.slice = l.slice
                           AND s.segment = l.segment
                           AND s.userid > 1
                           AND s.perm_table_name NOT IN ('Internal Worktable', 'S3') 
                     LEFT JOIN stl_query AS q
                            ON q.query = l.query
                           AND q.xid = l.xid
                           AND q.userid > 1
                   WHERE l.userid > 1
                   AND   TRIM(s.perm_table_name) IS NOT NULL
                   AND   l.event_time >= dateadd(DAY,- 7,CURRENT_DATE)
                   AND   TRIM(split_part(l.event,':',1)) = 'Broadcasted a large number of rows across the network'
                   GROUP BY 1,
                            2) tab
               ON (tab.database_name = t.database
              AND tab.table_name = t.table)
  LEFT OUTER JOIN (SELECT q.database AS database_name,
                          TRIM(s.perm_table_name) AS table_name,
                          MAX(l.query) AS alert_sample_query,
                          COUNT(DISTINCT l.query) AS alert_query_count
                   FROM stl_alert_event_log AS l
                     LEFT JOIN stl_scan AS s
                            ON s.query = l.query
                           AND s.slice = l.slice
                           AND s.segment = l.segment
                           AND s.userid > 1
                           AND s.perm_table_name NOT IN ('Internal Worktable', 'S3') 
                     LEFT JOIN stl_query AS q
                            ON q.query = l.query
                           AND q.xid = l.xid
                           AND q.userid > 1
                   WHERE l.userid > 1
                   AND   TRIM(s.perm_table_name) IS NOT NULL
                   AND   l.event_time >= dateadd(DAY,- 7,CURRENT_DATE)
                   AND   TRIM(split_part(l.event,':',1)) = 'Scanned a large number of deleted rows'
                   GROUP BY 1,
                            2) tax
               ON (tax.database_name = t.database
              AND tax.table_name = t.table)
WHERE t."schema" NOT IN ('pg_internal','pg_catalog','pg_automv')
AND   t."schema" NOT LIKE 'pg_temp%'
ORDER BY tbl_rows DESC;"""

In [27]:
queries["AlterTableRecommendation"]="""SELECT r.type,
       trim(t.database_name) AS dbname,
       t.schema_name AS namespace,
       r.table_id,
       t.table_name,
       r.group_id,
       r.ddl,
       r.auto_eligible
FROM svv_alter_table_recommendations r
  INNER JOIN (SELECT DISTINCT(stv_tbl_perm.id) AS table_id
                    ,TRIM(pg_database.datname) AS database_name
                    ,TRIM(pg_namespace.nspname) AS schema_name
                    ,TRIM(relname) AS table_name
                FROM stv_tbl_perm
              INNER JOIN pg_database on pg_database.oid = stv_tbl_perm.db_id
              INNER JOIN pg_class on pg_class.oid = stv_tbl_perm.id
              INNER JOIN pg_namespace on pg_namespace.oid = pg_class.relnamespace
               WHERE schema_name NOT IN ('pg_internal', 'pg_catalog','pg_automv')) t
          ON (r.database = t.database_name
         AND r.table_id = t.table_id);"""

In [28]:
queries["MaterializedView"]="""WITH mv_info AS
(
  SELECT trim(db_name) AS dbname,
         trim("schema") AS namespace, 
         trim(name) AS mview_name,
         is_stale,
         state,
         CASE state
           WHEN 0 THEN 'The MV is fully recomputed when refreshed'
           WHEN 1 THEN 'The MV is incremental'
           WHEN 101 THEN 'The MV cant be refreshed due to a dropped column. This constraint applies even if the column isnt used in the MV'
           WHEN 102 THEN 'The MV cant be refreshed due to a changed column type. This constraint applies even if the column isnt used in the MV'
           WHEN 103 THEN 'The MV cant be refreshed due to a renamed table'
           WHEN 104 THEN 'The MV cant be refreshed due to a renamed column. This constraint applies even if the column isnt used in the MV'
           WHEN 105 THEN 'The MV cant be refreshed due to a renamed schema'
           ELSE NULL
         END AS state_desc,
         autorewrite,
         autorefresh
  FROM stv_mv_info
),
mv_state AS
(
  SELECT dbname,
         namespace,
         mview_name,
         state AS mv_state,
         event_desc,
         starttime AS event_starttime
  FROM (SELECT trim(db_name) AS dbname,
               trim(mv_schema) AS namespace, 
               trim(mv_name) AS mview_name,
               ROW_NUMBER() OVER (PARTITION BY db_name, mv_schema, mv_name ORDER BY starttime DESC) AS rnum,
               state,
               event_desc,
               starttime
        FROM stl_mv_state)
  WHERE rnum = 1
),
mv_ref_status AS
(
  SELECT dbname,
         refresh_db_username,
         namespace,
         mview_name,
         status AS refresh_status,
         refresh_type,
         starttime AS refresh_starttime,
         endtime AS refresh_endtime,
         datediff(ms, starttime, endtime)*1.0/1000 AS refresh_duration_secs
  FROM (SELECT trim(r.db_name) AS dbname,
               trim(r.schema_name) AS namespace, 
               pu.usename as refresh_db_username,
               trim(r.mv_name) AS mview_name,
               ROW_NUMBER() OVER (PARTITION BY r.db_name, r.schema_name, r.userid, r.mv_name ORDER BY starttime DESC) AS rnum,
               r.status,
               r.refresh_type,
               r.starttime,
               r.endtime
        FROM svl_mv_refresh_status r
        INNER JOIN pg_user pu on (r.userid = pu.usesysid))
  WHERE rnum = 1
)
SELECT i.*, 
       s.mv_state, s.event_desc, s.event_starttime,
       r.refresh_db_username, r.refresh_status, r.refresh_type, r.refresh_starttime, r.refresh_endtime, r.refresh_duration_secs
FROM mv_info i
  LEFT JOIN mv_state s ON (i.dbname = s.dbname AND i.namespace = s.namespace AND i.mview_name = s.mview_name)
  LEFT JOIN mv_ref_status r ON (i.dbname = r.dbname AND i.namespace = r.namespace AND i.mview_name = r.mview_name);"""

In [29]:
queries["Top50QueriesByRunTime"]="""SELECT TRIM(dbname) AS dbname,
       TRIM(db_username) AS db_username,
       MAX(SUBSTRING(replace(qrytext,chr (34),chr (92) + chr (34)),1,500)) AS qrytext,
       COUNT(query) AS num_queries,
       MIN(run_minutes) AS min_minutes,
       MAX(run_minutes) AS max_minutes,
       AVG(run_minutes) AS avg_minutes,
       SUM(run_minutes) AS total_minutes,
       SUM(compile_minutes) AS total_compile_minutes,
       SUM(num_compile_segments) AS total_num_compile_segments,
       MIN(query_temp_blocks_to_disk_mb) AS min_disk_spill_mb,
       MAX(query_temp_blocks_to_disk_mb) AS max_disk_spill_mb,
       AVG(query_temp_blocks_to_disk_mb) AS avg_disk_spill_mb,
       SUM(query_temp_blocks_to_disk_mb) AS total_disk_spill_mb,
       MAX(query) AS max_query_id,
       MAX(starttime)::DATE AS last_run,
       COUNT(DISTINCT starttime::DATE) AS num_days_executed,
       SUM(aborted) AS total_aborted,
       MAX(mylabel) qry_label,
       AVG(spectrum_object_count) AS avg_spectrum_object_used,
       AVG(federated_object_count) AS avg_federated_object_used,
       user_table_involved,
       TRIM(DECODE (event & 1,1,'Sortkey ','') || DECODE (event & 2,2,'Deletes ','') || DECODE (event & 4,4,'NL ','') || DECODE (event & 8,8,'Dist ','') || DECODE (event & 16,16,'Broacast ','') || DECODE (event & 32,32,'Stats ','')) AS Alert
FROM (SELECT stl_query.userid,
             pu.usename as db_username,
             label,
             stl_query.query,
             TRIM("DATABASE") AS dbname,
             NVL(qrytext_cur.text,TRIM(querytxt)) AS qrytext,
             MD5(NVL (qrytext_cur.text,TRIM(querytxt))) AS qry_md5,
             starttime,
             endtime,
             DATEDIFF(seconds,starttime,endtime)::NUMERIC(12,2) / 60 AS run_minutes,
             aborted,
             event,
             stl_query.label AS mylabel,
             CASE
               WHEN sqms.query_temp_blocks_to_disk IS NULL THEN 0
               ELSE sqms.query_temp_blocks_to_disk
             END AS query_temp_blocks_to_disk_mb,
             nvl(compile_secs,0)::NUMERIC(12,2) / 60 AS compile_minutes,
             nvl(num_compile_segments,0) AS num_compile_segments,
             s.user_table_involved,
             NVL(s3.spectrum_object_count,0) AS spectrum_object_count,
             NVL(f.federated_object_count,0) AS federated_object_count
      FROM stl_query
	  INNER JOIN pg_catalog.pg_user pu on (stl_query.userid = pu.usesysid)
        LEFT OUTER JOIN (SELECT query,
                                SUM(DECODE (TRIM(SPLIT_PART (event,':',1)),'Very selective query filter',1,'Scanned a large number of deleted rows',2,'Nested Loop Join in the query plan',4,'Distributed a large number of rows across the network',8,'Broadcasted a large number of rows across the network',16,'Missing query planner statistics',32,0)) AS event
                         FROM stl_alert_event_log
                         WHERE event_time >= DATEADD(DAY,-7,CURRENT_DATE)
                         GROUP BY query) AS alrt ON alrt.query = stl_query.query
        LEFT OUTER JOIN (SELECT ut.xid,
                                TRIM(SUBSTRING(text FROM STRPOS (UPPER(text),'SELECT'))) AS TEXT
                         FROM stl_utilitytext ut
                         WHERE SEQUENCE = 0
                         AND   text ilike 'DECLARE%'
                         GROUP BY text,
                                  ut.xid) qrytext_cur ON (stl_query.xid = qrytext_cur.xid)
        LEFT OUTER JOIN svl_query_metrics_summary sqms
                     ON (sqms.userid = stl_query.userid
                    AND sqms.query = stl_query.query)
        LEFT OUTER JOIN (SELECT userid,
                                xid,
                                pid,
                                query,
                                MAX(datediff (ms,starttime,endtime)*1.0 / 1000) AS compile_secs,
                                SUM(compile) AS num_compile_segments
                         FROM svcs_compile
                         GROUP BY userid,
                                  xid,
                                  pid,
                                  query) c
                     ON (c.userid = stl_query.userid
                    AND c.xid = stl_query.xid
                    AND c.pid = stl_query.pid
                    AND c.query = stl_query.query)
        LEFT OUTER JOIN (SELECT s.userid,
                                s.query,
                                LISTAGG(DISTINCT TRIM(s.perm_table_name),', ') AS user_table_involved
                         FROM stl_scan s
                             INNER JOIN (SELECT DISTINCT(stv_tbl_perm.id) AS table_id
                    ,TRIM(pg_database.datname) AS database_name
                    ,TRIM(pg_namespace.nspname) AS schema_name
                    ,TRIM(relname) AS table_name
                FROM stv_tbl_perm
              INNER JOIN pg_database on pg_database.oid = stv_tbl_perm.db_id
              INNER JOIN pg_class on pg_class.oid = stv_tbl_perm.id
              INNER JOIN pg_namespace on pg_namespace.oid = pg_class.relnamespace
               WHERE schema_name NOT IN ('pg_internal', 'pg_catalog','pg_automv')) t ON (s.tbl = t.table_id)
                         WHERE s.perm_table_name NOT IN ('Internal Worktable','S3')
                         AND   s.perm_table_name NOT LIKE ('volt_tt%')
                         AND   t.schema_name NOT IN ('pg_internal','pg_catalog')
                         GROUP BY s.userid,
                                  s.query) s
                     ON (stl_query.userid = s.userid
                    AND stl_query.query = s.query)
        LEFT OUTER JOIN (SELECT s.userid,
                                s.query,
                                COUNT(1) AS spectrum_object_count
                         FROM svl_s3query_summary s
                         WHERE s.external_table_name NOT IN ('PG Subquery')
                         GROUP BY s.userid,
                                  s.query) s3
                     ON (stl_query.userid = s3.userid
                    AND stl_query.query = s3.query)
        LEFT OUTER JOIN (SELECT f.userid,
                                f.query,
                                COUNT(1) AS federated_object_count
                         FROM svl_federated_query f
                         GROUP BY f.userid,
                                  f.query) f
                     ON (stl_query.userid = f.userid
                    AND stl_query.query = f.query)
      WHERE stl_query.userid <> 1
      AND   NVL(qrytext_cur.text,TRIM(querytxt)) NOT LIKE 'padb_fetch_sample:%'
      AND   NVL(qrytext_cur.text,TRIM(querytxt)) NOT LIKE 'CREATE TEMP TABLE volt_tt_%'
      AND   stl_query.starttime >= DATEADD(DAY,-7,CURRENT_DATE))
GROUP BY TRIM(dbname),
         TRIM(db_username),
         qry_md5,
         user_table_involved,
         event
ORDER BY avg_minutes DESC, num_queries DESC LIMIT 50;"""

In [30]:
queries["Top50QueriesByDiskSpill"]="""SELECT TRIM(dbname) AS dbname,
       TRIM(db_username) AS db_username,
       MAX(SUBSTRING(replace(qrytext,chr (34),chr (92) + chr (34)),1,500)) AS qrytext,
       COUNT(query) AS num_queries,	    
       MIN(run_minutes) AS min_minutes,
       MAX(run_minutes) AS max_minutes,
       AVG(run_minutes) AS avg_minutes,
       SUM(run_minutes) AS total_minutes,
       SUM(compile_minutes) AS total_compile_minutes,
       SUM(num_compile_segments) AS total_num_compile_segments,
       MIN(query_temp_blocks_to_disk_mb) AS min_disk_spill_mb,
       MAX(query_temp_blocks_to_disk_mb) AS max_disk_spill_mb,
       AVG(query_temp_blocks_to_disk_mb) AS avg_disk_spill_mb,
       SUM(query_temp_blocks_to_disk_mb) AS total_disk_spill_mb,
       MAX(query) AS max_query_id,
       MAX(starttime)::DATE AS last_run,
       COUNT(DISTINCT starttime::DATE) AS num_days_executed,		   
       SUM(aborted) as total_aborted,
       MAX(mylabel) qry_label,
	   AVG(spectrum_object_count) AS avg_spectrum_object_used,
	   AVG(federated_object_count) AS avg_federated_object_used,
	   user_table_involved,
       TRIM(DECODE (event & 1,1,'Sortkey ','') || DECODE (event & 2,2,'Deletes ','') || DECODE (event & 4,4,'NL ','') || DECODE (event & 8,8,'Dist ','') || DECODE (event & 16,16,'Broacast ','') || DECODE (event & 32,32,'Stats ','')) AS Alert
FROM (SELECT stl_query.userid,
             pu.usename as db_username,
             label,
             stl_query.query,
             TRIM("DATABASE") AS dbname,
             NVL(qrytext_cur.text,TRIM(querytxt)) AS qrytext,
             MD5(NVL (qrytext_cur.text,TRIM(querytxt))) AS qry_md5,
             starttime,
             endtime,
             DATEDIFF(seconds,starttime,endtime)::NUMERIC(12,2) / 60 AS run_minutes,
             aborted,
             event,
             stl_query.label AS mylabel,
             CASE
               WHEN sqms.query_temp_blocks_to_disk IS NULL THEN 0
               ELSE sqms.query_temp_blocks_to_disk
             END AS query_temp_blocks_to_disk_mb,
             nvl(compile_secs,0)::NUMERIC(12,2) / 60 AS compile_minutes,
             nvl(num_compile_segments,0) AS num_compile_segments,
			 s.user_table_involved,
			 NVL(s3.spectrum_object_count,0) AS spectrum_object_count,
			 NVL(f.federated_object_count,0) AS federated_object_count
        FROM stl_query
		INNER JOIN pg_catalog.pg_user pu on (stl_query.userid = pu.usesysid)
        LEFT OUTER JOIN (SELECT query,
                                SUM(DECODE (TRIM(SPLIT_PART (event,':',1)),'Very selective query filter',1,'Scanned a large number of deleted rows',2,'Nested Loop Join in the query plan',4,'Distributed a large number of rows across the network',8,'Broadcasted a large number of rows across the network',16,'Missing query planner statistics',32,0)) AS event
                         FROM stl_alert_event_log
                         WHERE event_time >= DATEADD(DAY,-7,CURRENT_DATE)
                         GROUP BY query) AS alrt ON alrt.query = stl_query.query
        LEFT OUTER JOIN (SELECT ut.xid,
                                TRIM(SUBSTRING(text FROM STRPOS (UPPER(text),'SELECT'))) AS TEXT
                         FROM stl_utilitytext ut
                         WHERE SEQUENCE = 0
                         AND   text ilike 'DECLARE%'
                         GROUP BY text,
                                  ut.xid) qrytext_cur ON (stl_query.xid = qrytext_cur.xid)
        LEFT OUTER JOIN svl_query_metrics_summary sqms
                     ON (sqms.userid = stl_query.userid AND sqms.query = stl_query.query)
        LEFT OUTER JOIN (SELECT userid,
                                xid,
                                pid,
                                query,
                                MAX(datediff (ms,starttime,endtime)*1.0 / 1000) AS compile_secs,
                                SUM(compile) AS num_compile_segments
                         FROM svcs_compile
                         GROUP BY userid,
                                  xid,
                                  pid,
                                  query) c
                     ON (c.userid = stl_query.userid
                    AND c.xid = stl_query.xid
                    AND c.pid = stl_query.pid
                    AND c.query = stl_query.query)
        LEFT OUTER JOIN (SELECT s.userid,
                                s.query,
                                LISTAGG(distinct TRIM(s.perm_table_name),', ') AS user_table_involved
                           FROM stl_scan s
                             INNER JOIN (SELECT DISTINCT(stv_tbl_perm.id) AS table_id
                    ,TRIM(pg_database.datname) AS database_name
                    ,TRIM(pg_namespace.nspname) AS schema_name
                    ,TRIM(relname) AS table_name
                FROM stv_tbl_perm
              INNER JOIN pg_database on pg_database.oid = stv_tbl_perm.db_id
              INNER JOIN pg_class on pg_class.oid = stv_tbl_perm.id
              INNER JOIN pg_namespace on pg_namespace.oid = pg_class.relnamespace
               WHERE schema_name NOT IN ('pg_internal', 'pg_catalog','pg_automv')) t ON (s.tbl = t.table_id)
                          WHERE s.perm_table_name NOT IN ('Internal Worktable','S3')
						    AND s.perm_table_name NOT LIKE ('volt_tt%')
                            AND t.schema_name NOT IN ('pg_internal', 'pg_catalog') 
                         GROUP BY s.userid,
                                  s.query) s ON (stl_query.userid = s.userid AND stl_query.query = s.query)	
        LEFT OUTER JOIN (SELECT s.userid,
                                s.query,
                                COUNT(1) AS spectrum_object_count 
                           FROM svl_s3query_summary s 
						  WHERE s.external_table_name NOT IN ('PG Subquery')
                         GROUP BY s.userid,
                                  s.query) s3 ON (stl_query.userid = s3.userid AND stl_query.query = s3.query)
        LEFT OUTER JOIN (SELECT f.userid,
                                f.query,
                                COUNT(1) AS federated_object_count 
                           FROM svl_federated_query f 
                         GROUP BY f.userid,
                                  f.query) f ON (stl_query.userid = f.userid AND stl_query.query = f.query)								  
      WHERE stl_query.userid <> 1
	    AND NVL(qrytext_cur.text,TRIM(querytxt)) NOT LIKE 'padb_fetch_sample:%' 
		AND NVL(qrytext_cur.text,TRIM(querytxt)) NOT LIKE 'CREATE TEMP TABLE volt_tt_%'
        AND stl_query.starttime >= DATEADD(DAY,-7,CURRENT_DATE))
GROUP BY TRIM(dbname),
         TRIM(db_username),
         qry_md5,
		 user_table_involved,
         event
ORDER BY avg_disk_spill_mb DESC, num_queries DESC LIMIT 50;"""

In [31]:
queries["CopyPerformance"]="""SELECT a.endtime::DATE AS copy_date,
       trim(d.region) AS aws_region,
       trim(d.s3_bucket) AS s3_bucket,
       trim(d.file_format) AS file_format,
       trim(q."database") as dbname,
       a.tbl AS table_id,
       trim(c.nspname) AS namespace,
       trim(b.relname) AS table_name,
       SUM(a.rows_inserted) AS rows_inserted,
       SUM(d.distinct_files) AS files_scanned,
       SUM(d.mb_scanned) AS mb_scanned,
       (SUM(d.distinct_files)::NUMERIC(19,3) / COUNT(DISTINCT a.query)::NUMERIC(19,3))::NUMERIC(19,3) AS avg_files_per_copy,
       (SUM(d.mb_scanned) / SUM(d.distinct_files)::NUMERIC(19,3))::NUMERIC(19,3) AS avg_file_size_mb,
       MAX(d.files_compressed) AS files_compressed,
       MAX(cluster_slice_count) AS cluster_slice_count,
       AVG(d.used_slice_count) AS avg_used_slice_count,
       COUNT(DISTINCT a.query) no_of_copy,
       MAX(a.query) AS sample_query,
       ROUND((SUM(d.mb_scanned)*1024 *1000000.0 / SUM(d.load_micro)),4) AS scan_rate_kbps,
       ROUND((SUM(a.rows_inserted)*1000000.0 / SUM(a.insert_micro)),4) AS insert_rate_rows_per_second,
       ROUND(SUM(d.copy_duration_micro)/1000000.0,4) AS total_copy_time_secs,
       ROUND(AVG(d.copy_duration_micro)/1000000.0,4) AS avg_copy_time_secs,
       ROUND(SUM(d.compression_micro)/1000000.0,4) AS total_compression_time_secs,
       ROUND(AVG(d.compression_micro)/1000000.0,4) AS avg_compression_time_secs,       
       SUM(d.total_transfer_retries) AS total_transfer_retries,
       SUM(d.distinct_error_files) AS distinct_error_files,
       SUM(d.load_error_count) AS load_error_count
FROM (SELECT query,
             tbl,
             SUM(ROWS) AS rows_inserted,
             MAX(endtime) AS endtime,
             datediff('microsecond',MIN(starttime),MAX(endtime)) AS insert_micro
      FROM stl_insert
      GROUP BY query,
               tbl) a,
     pg_class b,
     pg_namespace c,
     (SELECT b.region,
             l.file_format,
             b.query,
             b.bucket as s3_bucket,
             COUNT(DISTINCT b.bucket||b.key) AS distinct_files,
             COUNT(DISTINCT b.slice) as used_slice_count,
             SUM(b.transfer_size) / 1024 / 1024 AS mb_scanned,
             SUM(b.transfer_time) AS load_micro,
             SUM(b.compression_time) AS compression_micro,
             datediff('microsecond',MIN('2000-01-01'::timestamp + (start_time/1000000.0)* interval '1 second'),MAX('2000-01-01'::timestamp + (end_time/1000000.0)* interval '1 second')) AS copy_duration_micro,
             SUM(b.retries) as total_transfer_retries,
             SUM(nvl(se.distinct_error_files,0)) AS distinct_error_files,
             SUM(nvl(se.load_error_count,0)) AS load_error_count,
             CASE WHEN SUM(b.transfer_size) = SUM(b.data_size) then 'N' else  'Y' end AS files_compressed
      FROM stl_s3client b
      INNER JOIN (select userid, query, MAX(file_format) as file_format FROM stl_load_commits GROUP BY userid, query) l ON (l.userid = b.userid and l.query = b.query)
      LEFT OUTER JOIN (select userid, query, COUNT(DISTINCT bucket||key) AS distinct_error_files, COUNT(1) AS load_error_count FROM stl_s3client_error GROUP BY userid, query) se ON (se.userid = b.userid and se.query = b.query)
      WHERE b.http_method = 'GET'
      GROUP BY b.region,l.file_format,b.query,b.bucket) d,
     stl_query q,
     (SELECT COUNT(1) AS cluster_slice_count FROM stv_slices)
WHERE a.tbl = b.oid
AND   b.relnamespace = c.oid
AND   d.query = a.query
AND   a.query = q.query
AND   lower(q.querytxt) LIKE '%copy %'
GROUP BY 1,2,3,4,5,6,7,8
ORDER BY 9 DESC, 21 DESC, 1 DESC
 LIMIT 50;"""

In [32]:
queries["SpectrumPerformance"]="""SELECT trim(et.schemaname) AS namespace,
       trim(et.tablename) AS external_table_name,
       trim(lq.file_format) AS file_format,
       trim(l.s3_bucket) AS s3_bucket,
       CASE
         WHEN et.compressed = 1 THEN 'Y'
         ELSE 'N'
       END AS is_table_compressed,
       CASE
         WHEN lq.is_partitioned = 't' THEN 'Y'
         ELSE 'N'
       END AS is_table_partitioned,
       CASE
         WHEN l.avg_file_splits > 1 THEN 'Y'
         ELSE 'N'
       END AS is_file_spittable,
       MAX(nvl (ep.external_table_partition_count,0)) AS external_table_partition_count,
       COUNT(1) total_query_count,
       SUM(CASE WHEN lp.qualified_partitions < ep.external_table_partition_count THEN 1 ELSE 0 END) total_query_using_Partition_Pruning_count,
       ROUND(SUM(CASE WHEN lp.qualified_partitions < ep.external_table_partition_count THEN 1 ELSE 0 END) / COUNT(1)::NUMERIC(38,4)*100.0,4) AS pct_of_query_using_Partition_Pruning,
       nvl(AVG(CASE WHEN lp.qualified_partitions != 0 THEN lp.qualified_partitions END),0) AS avg_Qualified_Partitions,
       nvl(AVG(CASE WHEN lp.qualified_partitions != 0 THEN lp.avg_assigned_partitions END),0) AS avg_Assigned_Partitions,
       ROUND(nvl(AVG(CASE WHEN lp.qualified_partitions != 0 THEN lq.avg_request_parallelism END),0),4) AS avg_Parallelism,
       nvl(AVG(CASE WHEN lp.qualified_partitions != 0 THEN lq.files END),0) AS avg_Files,
       AVG(lq.splits) AS avg_Split,
       ROUND(AVG(l.avg_max_file_size_mb),4) AS avg_max_file_size_mb,
       ROUND(AVG(l.avg_file_size_mb),4) AS avg_file_size_mb,
       ROUND(AVG(elapsed / 1000000::NUMERIC(38,4)),4) avg_Elapsed_sec,
       ROUND(SUM(elapsed / 1000000::NUMERIC(38,4)),4) Total_Elapsed_sec,
       SUM(nvl(r.spectrum_scan_error_count,0)) AS total_spectrum_scan_error_count,
       AVG(nvl(r.spectrum_scan_error_count,0)) AS avg_spectrum_scan_error_count,
       SUM(CASE WHEN lp.qualified_partitions = 0 THEN 1 ELSE 0 END) Queries_Using_No_S3Files
FROM svl_s3query_summary lq
  INNER JOIN stl_query q
          ON (q.userid = lq.userid
         AND q.query = lq.query
         AND q.xid = lq.xid
         AND q.pid = lq.pid)
  INNER JOIN svv_external_tables et ON (q.database || '_' || et.schemaname || '_' || et.tablename = replace (replace (lq.external_table_name,'S3 Scan ',''),'S3 Subquery ',''))
  LEFT OUTER JOIN (SELECT schemaname,
                          tablename,
                          COUNT(1) AS external_table_partition_count
                   FROM svv_external_partitions
                   GROUP BY schemaname,
                            tablename) ep
               ON (ep.schemaname = et.schemaname
              AND ep.tablename = et.tablename)
  LEFT OUTER JOIN svl_s3partition_summary lp ON lq.query = lp.query
  LEFT OUTER JOIN (SELECT query,
                          bucket AS s3_bucket,
                          AVG(max_file_size / 1000000.0) AS avg_max_file_size_mb,
                          AVG(avg_file_size / 1000000.0) AS avg_file_size_mb,
                          AVG(generated_splits) AS avg_file_splits
                   FROM svl_s3list
                   GROUP BY query,
                            bucket) l ON (lq.query = l.query)
  LEFT OUTER JOIN (SELECT query,
                          userid,
                          count(1) AS spectrum_scan_error_count
                   FROM svl_spectrum_scan_error
                   GROUP BY query, userid) r ON (lq.query = r.query and lq.userid = r.userid)
WHERE lq.starttime >= dateadd(day,- 7,CURRENT_DATE)
AND   lq.aborted = 0
GROUP BY 1,2,3,4,5,6,7
ORDER BY Total_Elapsed_sec DESC LIMIT 50;"""

In [33]:
queries["DataShareProducerObject"]="""WITH shared_table AS
(
  SELECT t.schema_name|| '.' ||t.table_name AS object_name,
         MAX(t.table_rows) AS table_rows,
         MAX(v.eventtime) AS last_vacuum_date,
         MAX(CASE WHEN v.is_recluster = 0 THEN 'N' WHEN v.is_recluster = 1 THEN 'Y' ELSE NULL END) AS is_last_vacuum_recluster,
         MAX(i.num_insert_operation) AS num_insert_operation,
         MAX(i.total_inserted_rows) AS total_inserted_rows,
         MAX(i.last_insert_date) AS last_insert_date,
         MAX(d.num_delete_operation) AS num_delete_operation,
         MAX(d.total_deleted_rows) AS total_deleted_rows,
         MAX(d.last_delete_date) AS last_delete_date
  FROM (SELECT DISTINCT (stv_tbl_perm.id) table_id,
               TRIM(pg_database.datname) AS "database",
               TRIM(pg_namespace.nspname) AS schema_name,
               TRIM(relname) AS table_name,
               reltuples::bigint AS table_rows
        FROM stv_tbl_perm
          JOIN pg_database ON pg_database.oid = stv_tbl_perm.db_id
          JOIN pg_class ON pg_class.oid = stv_tbl_perm.id
          JOIN pg_namespace ON pg_namespace.oid = pg_class.relnamespace
        WHERE schema_name NOT IN ('pg_internal','pg_catalog','pg_automv')) t
    LEFT OUTER JOIN stl_vacuum v
                 ON (t.table_id = v.table_id
                AND v.status NOT LIKE 'Skip%')
    LEFT OUTER JOIN (SELECT tbl,
                            COUNT(1) AS num_insert_operation,
                            SUM(ROWS) AS total_inserted_rows,
                            MAX(endtime) AS last_insert_date
                     FROM stl_insert
                     GROUP BY 1) i ON (i.tbl = t.table_id)
    LEFT OUTER JOIN (SELECT tbl,
                            COUNT(1) AS num_delete_operation,
                            SUM(ROWS) AS total_deleted_rows,
                            MAX(endtime) AS last_delete_date
                     FROM stl_delete
                     GROUP BY 1) d ON (d.tbl = t.table_id)
  GROUP BY 1
)
SELECT d.share_type,
       d.share_name,
       case when d.include_new = true then 'True' when d.include_new = false then 'False' else null end as include_new,
       d.producer_account,
       d.object_type,
       replace(substring(d.object_name,1,strpos (d.object_name,'.')),'.','') AS namespace,
       substring(d.object_name,strpos (d.object_name,'.') +1) AS object_name,
       ti.table_rows,
       ti.last_vacuum_date,
       ti.is_last_vacuum_recluster,
       ti.num_insert_operation,
       ti.total_inserted_rows,
       ti.last_insert_date,
       ti.num_delete_operation,
       ti.total_deleted_rows,
       ti.last_delete_date,
       CASE
         WHEN mi.is_stale = 't' THEN 'Y'
         WHEN mi.is_stale = 'f' THEN 'N'
         ELSE NULL
       END AS is_mv_stale,
       CASE
         WHEN mi.state = 1 THEN 'Y'
         WHEN mi.state <> 1 THEN 'N'
         ELSE NULL
       END AS is_mv_incremental_refresh,
       CASE
         WHEN mi.autorefresh = 1 THEN 'Y'
         WHEN mi.autorefresh <> 1 THEN 'N'
         ELSE NULL
       END AS is_mv_auto_refresh
FROM svv_datashare_objects d
  LEFT OUTER JOIN shared_table ti ON (d.object_name = ti.object_name)
  LEFT OUTER JOIN stv_mv_info mi ON (d.object_name = mi.schema|| '.' ||mi.name)
WHERE d.share_type = 'OUTBOUND'
ORDER BY 2,3,5,4;"""

In [34]:
queries["DataShareConsumerUsage"]="""with consumer_activity as 
(
select uc.userid
      ,u.usename as db_username
      ,uc.pid
      ,uc.xid
      ,min(uc.recordtime) as request_start_date
      ,max(uc.recordtime) as request_end_date
      ,datediff('milliseconds',min(uc.recordtime),max(uc.recordtime))::NUMERIC(38,4) / 1000 as request_duration_secs
      ,nvl(count(distinct uc.transaction_uid),0) as unique_transaction
      ,nvl(count(uc.request_id),0) as total_usage_consumer_count
      ,sum(case when trim(uc.error) = '' then 0 else 1 end) as request_error_count
  from svl_datashare_usage_consumer uc
  inner join pg_user u on (u.usesysid = uc.userid)
group by 1,2,3,4
)
,consumer_query as (
select trim(q."database") as dbname
      ,trim(cu.db_username) as db_username
      ,cu.request_start_date::date as request_date
      ,cu.request_duration_secs
      ,datediff('milliseconds',cu.request_end_date,q.starttime)::NUMERIC(38,4) / 1000  as request_interval_secs
      ,datediff('milliseconds',q.starttime,q.endtime)::NUMERIC(38,4) / 1000 as query_execution_secs
      ,datediff('milliseconds',request_start_date,q.endtime)::NUMERIC(38,4) / 1000 as total_execution_secs
      ,q.query
      ,cu.unique_transaction
      ,cu.total_usage_consumer_count
      ,cu.request_error_count      
from consumer_activity cu 
inner join stl_query q on (q.xid = cu.xid and q.pid = cu.pid and q.userid = cu.userid)
)
,consumer_query_aggregate as (
select cq.request_date
      ,cq.dbname
      ,cq.db_username
	  ,avg(cq.request_duration_secs) as avg_request_duration_secs
      ,sum(cq.request_duration_secs) as total_request_duration_secs
	  ,avg(cq.request_interval_secs) as avg_request_interval_secs
      ,sum(cq.request_interval_secs) as total_request_interval_secs
	  ,avg(cq.query_execution_secs) as avg_query_execution_secs
      ,sum(cq.query_execution_secs) as total_query_execution_secs
	  ,avg(cq.total_execution_secs) as avg_execution_secs
      ,sum(cq.total_execution_secs) as total_execution_secs
      ,count(cq.query) as query_count
      ,sum(cq.unique_transaction) as total_unique_transaction
      ,sum(cq.total_usage_consumer_count) as total_usage_consumer_count
      ,sum(cq.request_error_count) as total_request_error_count 
  from consumer_query cq
group by 1,2,3
)
,consumer_query_request_percentile AS (
SELECT cq.request_date
      ,cq.dbname
      ,cq.db_username
	  ,percentile_cont(0.8) within GROUP ( ORDER BY request_duration_secs) AS p80_request_sec
	  ,percentile_cont(0.9) within GROUP ( ORDER BY request_duration_secs) AS p90_request_sec
	  ,percentile_cont(0.99) within GROUP ( ORDER BY request_duration_secs) AS p99_request_sec
  from consumer_query cq
group by 1,2,3
)
select cqa.request_date
      ,cqa.dbname
      ,cqa.db_username
      ,cqa.query_count
	  ,cqa.avg_query_execution_secs
      ,cqa.total_query_execution_secs
	  ,cqa.avg_execution_secs
      ,cqa.total_execution_secs
	  ,cqa.avg_request_duration_secs
	  ,cqrp.p80_request_sec
	  ,cqrp.p90_request_sec
	  ,cqrp.p99_request_sec
      ,cqa.total_request_duration_secs
	  ,cqa.avg_request_interval_secs
      ,cqa.total_request_interval_secs	  
      ,cqa.total_unique_transaction
      ,cqa.total_usage_consumer_count
      ,cqa.total_request_error_count
  from consumer_query_aggregate cqa
  inner join consumer_query_request_percentile cqrp on (cqa.request_date = cqrp.request_date and cqa.dbname = cqrp.dbname and cqa.db_username = cqrp.db_username)
order by 1,2,3;"""

In [35]:
queries["ATOWorkerActions"]="""WITH latest_worker_action AS
(
  SELECT table_id,
         TYPE,
         trim(status) as status,
         eventtime,
         ROW_NUMBER() OVER (PARTITION BY table_id,TYPE ORDER BY eventtime DESC) AS rnum
  FROM svl_auto_worker_action w
)
SELECT trim(t.database_name) AS dbname,
       t.schema_name AS namespace,
       t.table_name,
       w.type,
       w.status AS latest_status,
       w.eventtime
FROM latest_worker_action w
  INNER JOIN (SELECT DISTINCT(stv_tbl_perm.id) AS table_id
                    ,TRIM(pg_database.datname) AS database_name
                    ,TRIM(pg_namespace.nspname) AS schema_name
                    ,TRIM(relname) AS table_name
                FROM stv_tbl_perm
              INNER JOIN pg_database on pg_database.oid = stv_tbl_perm.db_id
              INNER JOIN pg_class on pg_class.oid = stv_tbl_perm.id
              INNER JOIN pg_namespace on pg_namespace.oid = pg_class.relnamespace
               WHERE schema_name NOT IN ('pg_internal', 'pg_catalog','pg_automv')) t
          ON (t.table_id = w.table_id AND w.rnum = 1)
ORDER BY 1,2,3,4;"""

In [36]:
queries["WorkloadEvaluation"] = {}
queries["WorkloadEvaluation"]["text"] ="""WITH hour_list AS
(
  SELECT DATE_TRUNC('m',starttime) start_hour,
         dateadd('m',1,start_hour) AS end_hour
  FROM stl_query q
  WHERE starttime >= (getdate() -7)
  GROUP BY 1
),
scan_sum AS
(
  SELECT query,
         segment,
         SUM(bytes) AS bytes
  FROM stl_scan
  WHERE userid > 1
  GROUP BY query,
           segment
),
scan_list AS
(
  SELECT query,
         MAX(bytes) AS max_scan_bytes
  FROM scan_sum
  GROUP BY query
),
query_list AS
(
  SELECT w.query,
         exec_start_time,
         exec_end_time,
         ROUND(total_exec_time / 1000 / 1000.0,3) AS exec_sec,
         max_scan_bytes,
         CASE
           WHEN max_scan_bytes < 100000000 THEN 'small'
           WHEN max_scan_bytes BETWEEN 100000000 AND 500000000000 THEN 'medium'
           WHEN max_scan_bytes > 500000000000 THEN 'large'
         END AS size_type
  FROM stl_wlm_query w,
       scan_list sc
  WHERE sc.query = w.query
),
workload_exec_seconds  AS
(
select 
count(*) as query_cnt, 
SUM(CASE WHEN size_type = 'small' THEN  exec_sec  ELSE 0 END) AS small_workload_exec_sec_sum,
SUM(CASE WHEN size_type = 'medium' THEN  exec_sec  ELSE 0 END) AS medium_workload_exec_sec_sum,
SUM(CASE WHEN size_type = 'large' THEN  exec_sec  ELSE 0 END) AS large_workload_exec_sec_sum,
  
AVG(CASE WHEN size_type = 'small' THEN  exec_sec  ELSE 0 END) AS small_workload_exec_sec_avg,
AVG(CASE WHEN size_type = 'medium' THEN  exec_sec  ELSE 0 END) AS medium_workload_exec_sec_avg,
AVG(CASE WHEN size_type = 'large' THEN  exec_sec  ELSE 0 END) AS large_workload_exec_sec_avg,
  
MAX(CASE WHEN size_type = 'small' THEN  exec_sec  ELSE 0 END) AS small_workload_exec_sec_max,
MAX(CASE WHEN size_type = 'medium' THEN  exec_sec  ELSE 0 END) AS medium_workload_exec_sec_max,
MAX(CASE WHEN size_type = 'large' THEN  exec_sec  ELSE 0 END) AS large_workload_exec_sec_max,
  
  MIN(CASE WHEN size_type = 'small' THEN  exec_sec  ELSE 0 END) AS small_workload_exec_sec_min,
MIN(CASE WHEN size_type = 'medium' THEN  exec_sec  ELSE 0 END) AS medium_workload_exec_sec_min,
MIN(CASE WHEN size_type = 'large' THEN  exec_sec  ELSE 0 END) AS large_workload_exec_sec_min,
  
AVG(CASE WHEN size_type = 'small' THEN  max_scan_bytes  ELSE 0 END) AS small_workload_max_scan_bytes_avg,
AVG(CASE WHEN size_type = 'medium' THEN  max_scan_bytes  ELSE 0 END) AS medium_workload_max_scan_bytes_avg,
AVG(CASE WHEN size_type = 'large' THEN  max_scan_bytes  ELSE 0 END) AS large_workload_max_scan_bytes_avg,

  (small_workload_exec_sec_sum+medium_workload_exec_sec_sum+large_workload_exec_sec_sum) as total_workload_exec_sec_sum,
small_workload_exec_sec_sum/(total_workload_exec_sec_sum*1.00) as Small_workload_perc,
medium_workload_exec_sec_sum/(total_workload_exec_sec_sum*1.00) as Medium_workload_perc,
large_workload_exec_sec_sum/(total_workload_exec_sec_sum*1.00) as Large_workload_perc
from query_list
)
,query_list_2 AS
(
  SELECT start_hour,
         query,
         size_type,
         max_scan_bytes,
         exec_sec,
         exec_start_time,
         exec_end_time
  FROM hour_list h,
       query_list q
  WHERE exec_start_time BETWEEN start_hour AND end_hour
  OR    exec_end_time BETWEEN start_hour AND end_hour
  OR    (exec_start_time < start_hour AND exec_end_time > end_hour)
)

,hour_list_agg AS
(
  SELECT start_hour,
         SUM(CASE WHEN size_type = 'small' THEN 1 ELSE 0 END) AS small_query_cnt,
         SUM(CASE WHEN size_type = 'medium' THEN 1 ELSE 0 END) AS medium_query_cnt,
         SUM(CASE WHEN size_type = 'large' THEN 1 ELSE 0 END) AS large_query_cnt,
         COUNT(*) AS tot_query_cnt
  FROM query_list_2
  GROUP BY start_hour
) 
,utilization_perc AS
(
SELECT trunc(start_hour) AS sample_date,
       ROUND(100*SUM(CASE WHEN tot_query_cnt > 0 THEN 1 ELSE 0 END) / 1440.0,1) AS all_query_activite_perc,
       ROUND(100*SUM(CASE WHEN small_query_cnt > 0 THEN 1 ELSE 0 END) / 1440.0,1) AS small_query_activite_perc,
       ROUND(100*SUM(CASE WHEN medium_query_cnt > 0 THEN 1 ELSE 0 END) / 1440.0,1) AS medium_query_activite_perc,
       ROUND(100*SUM(CASE WHEN large_query_cnt > 0 THEN 1 ELSE 0 END) / 1440.0,1) AS large_query_activite_perc,
       MIN(start_hour) AS start_hour,
       MAX(start_hour) AS end_hour
FROM hour_list_agg
GROUP BY 1
)

,activity_perc as 
(
Select avg(small_query_activite_perc) AS AVG_small_query_activity_perc, 
avg(medium_query_activite_perc) AS AVG_medium_query_activity_perc,
avg(large_query_activite_perc) AS AVG_large_query_activity_perc
FROM utilization_perc
)

,mincount AS
(
SELECT trunc(start_hour) AS sample_date,
       SUM(CASE WHEN tot_query_cnt > 0 THEN 1 ELSE 0 END) AS tot_query_minute,
       SUM(CASE WHEN small_query_cnt > 0 THEN 1 ELSE 0 END) AS small_query_minute,
       SUM(CASE WHEN medium_query_cnt > 0 THEN 1 ELSE 0 END) AS medium_query_minute,
       SUM(CASE WHEN large_query_cnt > 0 THEN 1 ELSE 0 END) AS large_query_minute,
       MIN(start_hour) AS start_hour,
       MAX(start_hour) AS end_hour
FROM hour_list_agg
GROUP BY 1
),avgmincount AS
(
Select avg(small_query_minute) avg_small_query_minute, avg(medium_query_minute) avg_medium_query_minute, avg(large_query_minute) avg_large_query_minute
  from mincount
)
,final_output AS 
(
Select   small_workload_perc , medium_workload_perc,    large_workload_perc,    
  avg_small_query_activity_perc,    avg_medium_query_activity_perc,    avg_large_query_activity_perc,    
  avg_small_query_minute,    avg_medium_query_minute,    avg_large_query_minute,
  
  small_workload_exec_sec_avg, medium_workload_exec_sec_avg, large_workload_exec_sec_avg,
  small_workload_exec_sec_max, medium_workload_exec_sec_max, large_workload_exec_sec_max,
  small_workload_exec_sec_min, medium_workload_exec_sec_min, large_workload_exec_sec_min,
   total_query_cnt, total_small_query_cnt, 
  total_medium_query_cnt,total_large_query_cnt, 
  small_workload_max_scan_bytes_avg, 
  medium_workload_max_scan_bytes_avg, 
  large_workload_max_scan_bytes_avg
from activity_perc a, avgmincount b, workload_exec_seconds c, (  select count(*) as total_query_cnt, sum(case when size_type = 'small' then 1 else 0 end) as  total_small_query_cnt, 
                                                              sum(case when size_type = 'medium' then 1 else 0 end) as  total_medium_query_cnt, 
                                                              sum(case when size_type = 'large' then 1 else 0 end) as  total_large_query_cnt
                                                              from query_list )  d
where 1=1

)
select workloadtype, (perc_of_total_workload*100.00) as perc_of_total_workload, perc_duration_in_day, Total_query_minutes_in_day 
,workload_exec_sec_avg, workload_exec_sec_min, workload_exec_sec_max,query_cnt,scan_bytes_avg
from
(
select 
'Small' as workloadtype,
small_workload_perc as perc_of_total_workload,
avg_small_query_activity_perc as perc_duration_in_day,
avg_small_query_minute as Total_query_minutes_in_day, 
 small_workload_exec_sec_avg as  workload_exec_sec_avg,
  small_workload_exec_sec_max as  workload_exec_sec_max, 
   small_workload_exec_sec_min as  workload_exec_sec_min,
 total_small_query_cnt as query_cnt, 
  small_workload_max_scan_bytes_avg as scan_bytes_avg,
 1 as id
from final_output
union
select 
'Meduim' as workloadtype,
medium_workload_perc as perc_of_total_workload,
avg_medium_query_activity_perc as perc_duration_in_day,
avg_medium_query_minute as Total_query_minutes_in_day ,
  medium_workload_exec_sec_avg as  workload_exec_sec_avg,
    medium_workload_exec_sec_max as  workload_exec_sec_max, 
   medium_workload_exec_sec_min as  workload_exec_sec_min,
 total_medium_query_cnt as query_cnt, 
  medium_workload_max_scan_bytes_avg as scan_bytes_avg,
  2 as id
from final_output
union
select 
'Large' as workloadtype,
large_workload_perc as perc_of_total_workload,
avg_large_query_activity_perc as perc_duration_in_day,
avg_large_query_minute as Total_query_minutes_in_day, 
  large_workload_exec_sec_avg as  workload_exec_sec_avg,
   large_workload_exec_sec_max as  workload_exec_sec_max, 
   large_workload_exec_sec_min as  workload_exec_sec_min,
 total_large_query_cnt as query_cnt, 
  large_workload_max_scan_bytes_avg as scan_bytes_avg,
  3 as id
 from final_output
  ) a order by id asc;"""

# Prompting models with Queries

In [40]:
!pip install --upgrade -q boto3
!pip install --upgrade -q botocore

In [41]:
import json, boto3

session = boto3.session.Session()
session.region_name

'us-east-1'

In [42]:

bedrock = session.client(
    service_name="bedrock",
    region_name=session.region_name
)

In [43]:
bedrock.list_foundation_models()['modelSummaries']

[{'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-tg1-large',
  'modelId': 'amazon.titan-tg1-large',
  'modelName': 'Titan Text Large',
  'providerName': 'Amazon',
  'inputModalities': ['TEXT'],
  'outputModalities': ['TEXT'],
  'responseStreamingSupported': True,
  'customizationsSupported': [],
  'inferenceTypesSupported': ['ON_DEMAND'],
  'modelLifecycle': {'status': 'ACTIVE'}},
 {'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-image-generator-v1:0',
  'modelId': 'amazon.titan-image-generator-v1:0',
  'modelName': 'Titan Image Generator G1',
  'providerName': 'Amazon',
  'inputModalities': ['TEXT', 'IMAGE'],
  'outputModalities': ['IMAGE'],
  'customizationsSupported': ['FINE_TUNING'],
  'inferenceTypesSupported': ['PROVISIONED'],
  'modelLifecycle': {'status': 'ACTIVE'}},
 {'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-image-generator-v1',
  'modelId': 'amazon.titan-image-generator-v1',
  'modelName': 'Titan Image 

In [44]:
# For inference
bedrock_inference = session.client(
    service_name="bedrock-runtime", region_name=session.region_name
)

## Claude 3 Sonnet

In [65]:
def send_prompt(prompt_data, temperature=1.0, top_k=250, top_p=0.999, max_token_count=5000):

    body = json.dumps(
    {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": max_token_count,
        "temperature": temperature,
        "top_p": top_p,
        "top_k": top_k,
        "messages": [
          {
            "role": "user",
            "content": [
              {
                "type": "text",
                "text": f"""{prompt_data}"""
              }
            ]
            }
        ]
    })

    modelId = "anthropic.claude-3-sonnet-20240229-v1:0"
    accept = "application/json"
    contentType = "application/json"

    response = bedrock_inference.invoke_model(
        body=body, modelId=modelId, accept=accept, contentType=contentType
    )

    response_body = json.loads(response["body"].read())

    return response_body["content"][0]["text"]

In [66]:
import json
query = queries["WorkloadEvaluation"]
result = send_prompt(f"""Please explain the SQL command. Explain the numbers appearing in the query. think step by step. <SQLcommand> {query}  </SQLcommand>""", max_token_count=5000)

In [67]:
print(result)

Sure, I can explain the SQL command step by step, including the numbers appearing in the query.

This is a complex SQL query that involves multiple common table expressions (CTEs) and subqueries. The query aims to analyze the workload of queries based on their scan size and execution time, categorizing them into small, medium, and large workloads.

1. `WITH` clause:
   - This clause is used to define common table expressions (CTEs), which are temporary result sets that can be referenced later in the query.

2. `hour_list` CTE:
   ```sql
   WITH hour_list AS
   (
     SELECT DATE_TRUNC('m', starttime) start_hour,
            dateadd('m', 1, start_hour) AS end_hour
     FROM stl_query q
     WHERE starttime >= (getdate() - 7)
     GROUP BY 1
   )
   ```
   - This CTE creates a list of hourly intervals for the past 7 days.
   - `DATE_TRUNC('m', starttime)` truncates the `starttime` to the nearest minute.
   - `dateadd('m', 1, start_hour)` adds 1 minute to the `start_hour` to get the `end_

In [ ]:
result = send_prompt(f"""Please explain the SQL command. Explain the numbers appearing in the query. think step by step. <SQLcommand> {query}  </SQLcommand>""", max_token_count=20000)

In [64]:
print(result)

Sure, let's break down the SQL query step by step:

1. `WITH hour_list AS (...)`: This is a common table expression (CTE) named `hour_list`. It generates a list of hourly intervals for the past 7 days by truncating the `starttime` from the `stl_query` table to the nearest hour and adding one hour to create the `end_hour`.

2. `scan_sum AS (...)`: Another CTE that calculates the sum of bytes scanned for each query and segment from the `stl_scan` table, excluding user IDs less than or equal to 1 (system processes).

3. `scan_list AS (...)`: This CTE selects the maximum bytes scanned for each query from the `scan_sum` CTE.

4. `query_list AS (...)`: This CTE combines information from `stl_wlm_query` and `scan_list` CTEs. It calculates the execution time, retrieves the maximum scan bytes, and categorizes queries into 'small', 'medium', or 'large' based on the scan bytes.

5. `workload_exec_seconds AS (...)`: This CTE aggregates and calculates various metrics related to execution time and s

In [73]:
len(result)/5

731.2

In [68]:
result = send_prompt(f"""Please explain the SQL command. Explain the numbers appearing in the query. <SQLcommand> {query}  </SQLcommand>""", max_token_count=5000)

In [70]:
print(result)

This SQL query is quite complex and involves several nested subqueries and Common Table Expressions (CTEs). Let me break it down and explain each part:

1. **WITH hour_list AS**: This is a CTE that generates a list of hourly intervals for the past 7 days. It truncates the time to the start of the hour and calculates the end of the hour by adding 1 minute. This is used later to associate queries with the hourly intervals they ran in.

2. **scan_sum AS**: This CTE sums up the bytes scanned by each query and segment from the `stl_scan` table, grouping by query and segment. This is used to calculate the maximum scan bytes for each query.

3. **scan_list AS**: This CTE finds the maximum scan bytes for each query by taking the maximum value from the `scan_sum` CTE.

4. **query_list AS**: This CTE joins the `stl_wlm_query` table with the `scan_list` CTE to get information about each query, including start and end times, execution time, maximum scan bytes, and a size classification based on th

In [ ]:
import json
query = queries["WorkloadEvaluation"]
result = send_prompt(f"""Please explain the SQL command. Explain the numbers appearing in the query.<SQLcommand> {query}  </SQLcommand>""")

In [62]:
print(result)

The provided SQL query is quite complex and consists of multiple sub-queries (common table expressions or CTEs) and a final SELECT statement. Let me break it down for you:

1. `WITH hour_list AS (...)`: This CTE generates a list of hourly time intervals for the last 7 days by truncating the `starttime` column from the `stl_query` table to the nearest hour and then adding 1 hour to create the `end_hour`.

2. `scan_sum AS (...)`: This CTE calculates the sum of bytes scanned for each query and segment from the `stl_scan` table, excluding user IDs less than or equal to 1.

3. `scan_list AS (...)`: This CTE finds the maximum bytes scanned for each query by grouping the results from the `scan_sum` CTE.

4. `query_list AS (...)`: This CTE joins the `stl_wlm_query` table with the `scan_list` CTE to retrieve various query-related metrics, such as execution times, maximum bytes scanned, and a size classification based on the maximum bytes scanned.

5. `workload_exec_seconds AS (...)`: This CTE c

In [ ]:
import json
query = queries["WorkloadEvaluation"]
result = send_prompt(f"""Please explain the SQL command. <SQLcommand> {query}  </SQLcommand>""")

In [60]:
print(result)

The provided SQL command is a complex query that performs several operations to analyze the workload and performance of queries in a database system, specifically in a Redshift environment.

Here's a breakdown of the different parts of the query:

1. **CTE (Common Table Expressions):**
   - `hour_list`: This CTE generates a list of hourly intervals by truncating the `starttime` column from the `stl_query` table to the nearest minute and adding one minute to get the end of the hour range. It groups the data by the truncated `starttime` and considers only queries within the last 7 days.
   - `scan_sum`: This CTE calculates the sum of bytes scanned for each query and segment combination from the `stl_scan` table, excluding user ID 1 (superuser).
   - `scan_list`: This CTE finds the maximum bytes scanned for each query by grouping the results from `scan_sum`.
   - `query_list`: This CTE joins the `stl_wlm_query` table with `scan_list` to get the execution time, maximum scan bytes, and a si

In [77]:
result = send_prompt(f"""Please explain the SQL command. Explain the numbers appearing in the query. Identify all system tables used and present this as CSV using <tables> table1, table2, ... </tables>   <SQLcommand> {query}  </SQLcommand>""", max_token_count=2000)

In [78]:
print(result)

The provided SQL command is a complex query that performs various calculations and aggregations to analyze and report on the workload and query activity in a database system, specifically for different query size types (small, medium, and large) based on their maximum scan bytes.

Explanation of the numbers appearing in the query:

1. The number 1 in `WHERE userid > 1` filters out system-generated queries and considers only user queries.
2. The numbers 100000000 and 500000000000 are used as thresholds to categorize queries as 'small', 'medium', or 'large' based on their maximum scan bytes.
3. The number 7 in `WHERE starttime >= (getdate() - 7)` selects queries executed within the last 7 days.
4. The numbers 1000 and 1000000.0 are used for converting execution times from microseconds to seconds.
5. The number 1440.0 represents the number of minutes in a day (60 minutes × 24 hours) and is used for calculating the percentage of query activity during the day.

The system tables used in the

In [12]:
prompt = f"""Please explain the SQL command <SQLcommand> {query}  </SQLcommand>"""
prompt

'Please explain the SQL command <SQLcommand> hohoh  </SQLcommand>'

In [8]:
parameters="""Randomness and diversity
Info
Temperature
1
Top P
0.999
Top K
250
Length
Info
Maximum Length
2000
Stop sequences
Add
Human:"""

In [9]:
prompt = """Please explain the SQL command <SQLcommand> SELECT IQ.*,
       (IQ.wlm_queue_time_ms/ IQ.wlm_start_commit_time_ms)*100.0::numeric(6,2) AS pct_wlm_queue_time,
       (IQ.exec_only_time_ms/ IQ.wlm_start_commit_time_ms)*100.0::numeric(6,2) AS pct_exec_only_time,
       (IQ.commit_queue_time_ms/ IQ.wlm_start_commit_time_ms)*100.0::numeric(6,2) pct_commit_queue_time,
       (IQ.commit_time_ms/ IQ.wlm_start_commit_time_ms)*100.0::numeric(6,2) pct_commit_time
FROM (SELECT TRUNC(b.starttime) AS DAY,
             d.service_class,
             rtrim(s.name) as queue_name,
             c.node,
             COUNT(DISTINCT c.xid) AS count_commit_xid,
             SUM(datediff ('microsec',d.service_class_start_time,c.endtime)*0.001)::numeric(38,4) AS wlm_start_commit_time_ms,
             SUM(datediff ('microsec',d.queue_start_time,d.queue_end_time)*0.001)::numeric(38,4) AS wlm_queue_time_ms,
             SUM(datediff ('microsec',b.starttime,b.endtime)*0.001)::numeric(38,4) AS exec_only_time_ms,
             SUM(datediff ('microsec',c.startwork,c.endtime)*0.001)::numeric(38,4) commit_time_ms,
             SUM(datediff ('microsec',DECODE(c.startqueue,'2000-01-01 00:00:00',c.startwork,c.startqueue),c.startwork)*0.001)::numeric(38,4) commit_queue_time_ms
      FROM stl_query b,
           stl_commit_stats c,
           stl_wlm_query d,
           stv_wlm_service_class_config s
      WHERE b.xid = c.xid
      AND   b.query = d.query
      AND   c.xid > 0
      AND d.service_class = s.service_class
      GROUP BY 1,2,3,4
      ORDER BY 1,2,3,4) IQ;  </SQLcommand>"""

In [10]:
answer="""The provided SQL statement is a complex query that performs several calculations and aggregations on data from different system tables in Amazon Redshift. Here's a breakdown of what the query does:

1. The subquery (inside the `FROM` clause) retrieves data from multiple system tables: `stl_query`, `stl_commit_stats`, `stl_wlm_query`, and `stv_wlm_service_class_config`. It performs the following operations:
   - Groups the data by `DAY` (truncated start time), `service_class`, `queue_name`, and `node`.
   - Counts the distinct `xid` values (transaction IDs) for committed queries.
   - Calculates the total time (in milliseconds) for various phases of query execution:
     - `wlm_start_commit_time_ms`: Time from service class start to query commit.
     - `wlm_queue_time_ms`: Time spent in the queue before execution.
     - `exec_only_time_ms`: Actual execution time of the query.
     - `commit_time_ms`: Time taken for the commit phase.
     - `commit_queue_time_ms`: Time spent in the commit queue before the commit phase.

2. The outer query selects all columns from the subquery (`IQ.*`) and calculates the following additional columns:
   - `pct_wlm_queue_time`: Percentage of time spent in the queue before execution, relative to the total time from service class start to commit.
   - `pct_exec_only_time`: Percentage of actual execution time relative to the total time from service class start to commit.
   - `pct_commit_queue_time`: Percentage of time spent in the commit queue before the commit phase, relative to the total time from service class start to commit.
   - `pct_commit_time`: Percentage of time spent in the commit phase, relative to the total time from service class start to commit.

The purpose of this query is to analyze the performance of queries executed in Amazon Redshift, particularly the time spent in various phases of execution, such as queuing, execution, and commit. The calculated percentage columns provide insights into the relative distribution of time across these phases, which can help identify potential bottlenecks or areas for optimization.

It's important to note that this query retrieves data from several system tables in Amazon Redshift, which are designed to provide detailed information about query execution, workload management, and system performance. The specific tables used in this query are:

- `stl_query`: Contains information about executed queries, including start and end times, query text, and other query-level metrics.
- `stl_commit_stats`: Provides commit-related statistics for queries, such as commit time and queue time.
- `stl_wlm_query`: Stores workload management-related data for queries, including service class information and queue times.
- `stv_wlm_service_class_config`: Contains configuration details for the defined service classes in the system.

By combining data from these tables, the query can provide a comprehensive view of query performance and resource utilization in Amazon Redshift."""

In [6]:
prompt_log={"model":model,"prompt":prompt,"answer":answer,"parameters":parameters,"query":}

In [ ]:
prompts

In [85]:
table1="""Column name  |  Data type  |  Description   
---|---|---  
event  |  character(50)  |  Connection or authentication event.   
record_time  |  timestamp  |  Time the event occurred.   
remote_host  |  character(45)  |  Name or IP address of remote host.   
remote_port  |  character(32)  |  Port number for remote host.   
session_id  |  integer  |  Process ID associated with the statement.   
database_name  |  character(50)  |  Database name.   
user_name  |  character(50)  |  Username.   
auth_method  |  character(32)  |  Authentication method.   
duration  |  integer  |  Duration of connection in microseconds.   
ssl_version  |  character(50)  |  Secure Sockets Layer (SSL) version.   
ssl_cipher  |  character(128)  |  SSL cipher.   
mtu  |  integer  |  Maximum transmission unit (MTU).   
ssl_compression  |  character(64)  |  SSL compression type.   
ssl_expansion  |  character(64)  |  SSL expansion type.   
iam_auth_guid  |  character(36)  |  The IAM authentication ID for the CloudTrail request.   
application_name  |  character(250)  |  The initial or updated name of the application for a session.   
driver_version  |  character(64)  |  The version of ODBC or JDBC driver that connects to your Amazon Redshift cluster from your third-party SQL client tools.   
os_version  |  character(64)  |  The version of the operating system that is on the client machine that connects to your Amazon Redshift cluster.   
plugin_name  |  character(32)  |  The name of the plugin used to connect to your Amazon Redshift cluster.   
protocol_version  |  integer  |  The internal protocol version that the Amazon Redshift driver uses when establishing its connection with the server. The protocol versions are negotiated between the driver and server. The version describes the features available. Valid values include: 0 (BASE_SERVER_PROTOCOL_VERSION) 1 (EXTENDED_RESULT_METADATA_SERVER_PROTOCOL_VERSION) â To save a round trip per query, the server sends extra result set metadata information. 2 (BINARY_PROTOCOL_VERSION) â Depending on the data type of the result set, the server sends data in binary format. 3 (EXTENDED2_RESULT_METADATA_SERVER_PROTOCOL_VERSION) â The server sends case sensitivity (collation) information of a column.   
global_session_id  |  character(36)  |  The globally unique identifier for the current session. The session ID persists through node failure restarts."""


In [80]:
table="""Column name  |  Data type  |  Description   
---|---|---  
share_id  |  integer  |  The object ID (OID) of the datashare.   
share_name  |  varchar(128)  |  The name of the datashare.   
request_id  |  varchar(50)  |  The unique ID of the requested API call.   
request_type  |  varchar(25)  |  The type of the request made to the producer cluster.   
object_type  |  varchar(64)  |  The type of the object being shared from the datashare. Possible values are schemas, tables, columns, functions, and views.   
object_oid  |  integer  |  The ID of the object being shared from the datashare.   
object_name  |  varchar(128)  |  The name of the object being shared from the datashare.   
consumer_account  |  varchar(16)  |  The account of the consumer account that the datashare is shared to.   
consumer_namespace  |  varchar(64)  |  The namespace of the consumer account that the datashare is shared to.   
consumer_transaction_uid  |  varchar(50)  |  The unique transaction ID of the statement on the consumer cluster.   
record_time  |  timestamp  |  The time when the action is recorded.   
status  |  integer  |  The status of the datashare.   
error_message  |  varchar(512)  |  The message for an error.   
consumer_region  |  char(64)  |  The Region that the consumer cluster is in. 
"""

In [81]:
import json
result = send_prompt(f"""Please write 10 informative troubleshooting SQL queries on the table described in markdown.<markdown> {table1}  </markdown>""", max_token_count=2000)

In [83]:
print(result)

1. `SELECT object_name, object_type FROM datashare_table WHERE object_type = 'tables';`
   This query retrieves the names and types of all objects that are tables from the datashare.

2. `SELECT DISTINCT consumer_account, consumer_namespace FROM datashare_table;`
   This query retrieves the distinct consumer accounts and namespaces that the datashares are shared with.

3. `SELECT share_name, COUNT(DISTINCT object_name) AS num_objects FROM datashare_table GROUP BY share_name;`
   This query retrieves the names of the datashares and the number of distinct objects shared in each datashare.

4. `SELECT request_type, COUNT(*) AS request_count FROM datashare_table GROUP BY request_type;`
   This query retrieves the different types of requests made to the producer cluster and the count of each request type.

5. `SELECT consumer_region, COUNT(*) AS region_count FROM datashare_table GROUP BY consumer_region;`
   This query retrieves the distinct consumer regions and the count of datashares shar

In [86]:
import json
result = send_prompt(f"""Please write 10 informative troubleshooting SQL queries on the table described in markdown.<markdown> {table1}  </markdown>""", max_token_count=2000)

In [87]:
print(result)

Here are 10 informative troubleshooting SQL queries on the given table:

1. **Identify slow connections**
```sql
SELECT remote_host, duration, session_id
FROM table_name
ORDER BY duration DESC
LIMIT 10;
```
This query retrieves the top 10 longest connections based on the `duration` column, along with the `remote_host` and `session_id`. It can help identify slow connections and potential performance issues.

2. **Check for failed authentication attempts**
```sql
SELECT remote_host, user_name, auth_method, event, record_time
FROM table_name
WHERE event LIKE '%failed%'
ORDER BY record_time DESC
LIMIT 20;
```
This query retrieves the most recent 20 records where the `event` column contains the word "failed". It can help identify potential security issues or brute-force attacks.

3. **Monitor connections by user**
```sql
SELECT user_name, COUNT(*) AS connection_count
FROM table_name
GROUP BY user_name
ORDER BY connection_count DESC;
```
This query counts the number of connections per user a

In [89]:
table3="""Column name  |  Data type  |  Description   
---|---|---  
slice  |  integer  |  Node slice.   
col  |  integer  |  Zero-based index for the column. Every table you create has three hidden columns appended to it: INSERT_XID, DELETE_XID, and ROW_ID (OID). A table with 3 user-defined columns contains 6 actual columns, and the user-defined columns are internally numbered as 0, 1, and 2. The INSERT_XID, DELETE_XID, and ROW_ID columns are numbered 3, 4, and 5, respectively, in this example.   
tbl  |  integer  |  Table ID for the database table.   
blocknum  |  integer  |  ID for the data block.   
num_values  |  integer  |  Number of values contained on the block.   
extended_limits  |  integer  |  For internal use.   
minvalue  |  bigint  |  Minimum data value of the block. Stores first eight characters as 64-bit integer for non-numeric data. Used for disk scanning.   
maxvalue  |  bigint  |  Maximum data value of the block. Stores first eight characters as 64-bit integer for non-numeric data. Used for disk scanning.   
sb_pos  |  integer  |  Internal Amazon Redshift identifier for super block position on the disk.   
pinned  |  integer  |  Whether or not the block is pinned into memory as part of pre-load. 0 = false; 1 = true. Default is false.   
on_disk  |  integer  |  Whether or not the block is automatically stored on disk. 0 = false; 1 = true. Default is false.   
modified  |  integer  |  Whether or not the block has been modified. 0 = false; 1 = true. Default is false.   
hdr_modified  |  integer  |  Whether or not the block header has been modified. 0 = false; 1 = true. Default is false.   
unsorted  |  integer  |  Whether or not a block is unsorted. 0 = false; 1 = true. Default is true.   
tombstone  |  integer  |  For internal use.   
preferred_diskno  |  integer  |  Disk number that the block should be on, unless the disk has failed. Once the disk has been fixed, the block will move back to this disk.   
temporary  |  integer  |  Whether or not the block contains temporary data, such as from a temporary table or intermediate query results. 0 = false; 1 = true. Default is false.   
newblock  |  integer  |  Indicates whether or not a block is new (true) or was never committed to disk (false). 0 = false; 1 = true.   
num_readers  |  integer  |  Number of references on each block.   
flags  |  integer  |  Internal Amazon Redshift flags for the block header. 
"""

In [90]:
import json
result = send_prompt(f"""Please write 10 informative troubleshooting SQL queries on the table described in markdown.<markdown> {table3}  </markdown>""", max_token_count=2000)

In [91]:
print(result)

Here are 10 informative troubleshooting SQL queries on the described table:

1. **Find blocks with minimum or maximum values outside expected range**
```sql
SELECT slice, col, tbl, blocknum, minvalue, maxvalue
FROM svv_diskusage
WHERE minvalue < expected_min_value OR maxvalue > expected_max_value;
```

2. **Identify pinned blocks**
```sql
SELECT slice, col, tbl, blocknum
FROM svv_diskusage
WHERE pinned = 1;
```

3. **Find modified blocks**
```sql
SELECT slice, col, tbl, blocknum 
FROM svv_diskusage
WHERE modified = 1;
```

4. **Check for unsorted blocks**
```sql
SELECT slice, col, tbl, blocknum
FROM svv_diskusage  
WHERE unsorted = 1;
```

5. **Look for temporary blocks**
```sql
SELECT slice, col, tbl, blocknum
FROM svv_diskusage
WHERE temporary = 1;
```

6. **Identify new blocks not committed to disk**
```sql
SELECT slice, col, tbl, blocknum  
FROM svv_diskusage
WHERE newblock = 1;
```

7. **Find blocks with high number of readers**
```sql
SELECT slice, col, tbl, blocknum, num_readers

In [92]:
result = send_prompt(f"""For table STV_BLOCKLIST defined in markdown, write SQL query to shows whether or not table data is actually distributed over all slices.<markdown> {table3}  </markdown>""", max_token_count=2000)
print(result)

To check if the table data is distributed over all slices, you can use the following SQL query:

```sql
SELECT slice, COUNT(DISTINCT tbl) AS num_tables
FROM stv_blocklist
GROUP BY slice
ORDER BY slice;
```

This query will return the number of distinct tables present on each slice. If the data is distributed across all slices, you should see one row for each slice with a non-zero value in the `num_tables` column.

To confirm that the data is truly distributed evenly across all slices, you can compare the `num_tables` values across all slices. If they are relatively similar, it indicates that the data is distributed evenly. However, if you see significant differences in the `num_tables` values, it may suggest an uneven data distribution.

Additionally, you can check the total number of slices by running the following query:

```sql
SELECT COUNT(DISTINCT slice) AS num_slices
FROM stv_blocklist;
```

This will give you the total number of slices in your Amazon Redshift cluster. You can th

In [93]:
result = send_prompt(f"""For Redshift system table STV_BLOCKLIST defined in markdown, tell the story about what is in the table and why is this important.<markdown> {table3}  </markdown>""", max_token_count=2000)
print(result)

The `STV_BLOCKLIST` system table in Amazon Redshift provides detailed information about the data blocks that make up the tables in a Redshift cluster. Each row in this table represents a specific data block, and the columns provide various metadata about that block. Understanding the information in this table can be important for several reasons:

1. **Performance Optimization**: The `STV_BLOCKLIST` table can help identify potential performance bottlenecks related to data storage and access. For example, columns like `minvalue` and `maxvalue` can indicate if data is well-distributed across blocks, which can affect query performance. Additionally, columns like `pinned` and `on_disk` can provide insights into memory usage and disk I/O patterns.

2. **Data Distribution Analysis**: By examining the `num_values` column, you can determine how evenly data is distributed across blocks. Skewed data distribution can lead to query performance issues, and this table can help identify such imbalanc